# Import Library

In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast
from tabulate import tabulate
import torch

code_path = os.path.abspath('../')
sys.path.append(code_path)

from credentials import connection_string

In [2]:
database_name = "Karangasem"

# Import Data

## Connect to MongoDB Cluster

In [6]:
collection_name = f"Bali_{database_name}_Raw"

In [7]:
client = MongoClient(connection_string)
db = client[database_name]
collection = db[collection_name]

## Get All Documents from Selected Collection

In [8]:
all_documents = collection.find()

## Convert to DataFrame

In [9]:
df = pd.DataFrame(list(all_documents))

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [10]:
df

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,51/Pid.B/2024/PN Amp,Pencurian,"Jumat, 09 Agu. 2024",53 Hari,[I KOMANG ARIMBAWA Alias LELANG],[Ida Ayu Putu Widhiantini],"[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",5,[I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara...,M E N G A D I L I\nMenyatakan Terdakwa I Koman...,DAKWAAN :\n------ Bahwa ia Terdakwa I KOMANG A...
1,Minutasi,43/Pid.Sus/2024/PN Amp,Narkotika,"Kamis, 01 Agu. 2024",35 Hari,[MOH. SOLEH Alias SOLEH],"[Ardi Putra Dewa Agung, S.H]","[Meilia Christina Mulyaningrum~Hakim Ketua~Ya,...",2,[MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bu...,MENGADILI:\nMenyatakan Terdakwa Moh Soleh alia...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...
2,Minutasi,40/Pid.Sus/2024/PN Amp,Narkotika,"Rabu, 24 Jul. 2024",48 Hari,[SOIMUN Alias WAYAN],"[M. Thoriq Ardiansyah, S.H.]","[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",3,[SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan ...,MENGADILI:\n Menyatakan Terdakwa Soimun alias...,DAKWAAN :\n ------- Bahwa Terdakwa SOIMUN ali...
3,Minutasi,49/Pid.B/2024/PN Amp,Pencurian,"Kamis, 08 Agu. 2024",33 Hari,[I NENGAH SEMIARTA],"[Ardi Putra Dewa Agung, S.H]","[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",2,[I NENGAH SEMIARTA~Pidana Penjara Waktu Terten...,M E N G A D I L I\nMenyatakan Terdakwa I Nenga...,Dakwaan\nBahwa Terdakwa I Nengah Semiarta (yan...
4,Minutasi,45/Pid.B/2024/PN Amp,Pencurian,"Senin, 05 Agu. 2024",31 Hari,[I KETUT KERTA],[Ida Ayu Putu Widhiantini],"[Meilia Christina Mulyaningrum~Hakim Ketua~Ya,...",4,[I KETUT KERTA~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Ketut Kerta ...,DAKWAAN :\n------ Bahwa Terdakwa I KETUT KERTA...
...,...,...,...,...,...,...,...,...,...,...,...,...
487,Minutasi,26/Pid.B/2018/PN Amp,Penganiayaan,"Kamis, 22 Mar. 2018",42 Hari,[I MADE MERTA Als. DEGUL],"[I PUTU ERRYC SUNAS ARINTAMA,SH]","[Ni Made Kushandari~Hakim Ketua~Ya, I Gusti Ay...",5,[I MADE MERTA Als. DEGUL~Pidana Penjara Waktu ...,M E N G A D I L I :\nMenyatakan Terdakw...,Perbuatan terdakwa diatur dan diancam pidana d...
488,Minutasi,19/Pid.B/2018/PN Amp,Kejahatan yang Membahayakan Keamananan Umum Ba...,"Rabu, 21 Feb. 2018",64 Hari,"[I MADE SURI Als DEGDEG, I NYOMAN DUDUG, I MAD...","[NUR APRILIYANTO,SH]","[Ni Made Kushandari~Hakim Ketua~Ya, I Gusti Ay...",2,[I MADE SURI Als DEGDEG~Pidana Penjara Waktu T...,MENGADILI:\nMenyatakan TerdakwaI MADE SURI Als...,Bahwa terdakwa I MADE SURI Als DEGDEG bersama-...
489,Minutasi,17/Pid.B/2018/PN Amp,"Penadahan, Penerbitan, dan Pencetakan","Senin, 19 Feb. 2018",49 Hari,[I MADE SUGIARTA],"[NI KADEK DRIPTAYANTI,SH]","[Putu Ayu Sudariasih~Hakim Ketua~Ya, I Gusti P...",1,[I MADE SUGIARTA~Pidana Penjara Waktu Tertentu...,-------------------------------------------- ...,---------- Perbuatan terdakwa sebagaimana diat...
490,Minutasi,112/Pid.B/2017/PN Amp,Pencurian,"Jumat, 22 Des. 2017",66 Hari,[I NENGAH APRI AGUSTINA ALS. degle als. GRANDONG],"[PUTU YUMI ANTARI, SH.]","[Putu Ayu Sudariasih~Hakim Ketua~Ya, Ni Made K...",5,[I NENGAH APRI AGUSTINA ALS. degle als. GRANDO...,M E N G A D I L I :\nMenyatakan Terdakwa I NEN...,Bahwa ia terdakwa I NENGAH APRI AGUSTINA ALS. ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status_perkara       492 non-null    object
 1   nomor_perkara        492 non-null    object
 2   klasifikasi_perkara  492 non-null    object
 3   tanggal_pendaftaran  492 non-null    object
 4   lama_proses          492 non-null    object
 5   terdakwa             492 non-null    object
 6   penuntut_umum        492 non-null    object
 7   hakim                492 non-null    object
 8   jumlah_saksi         492 non-null    int64 
 9   putusan_hukuman      492 non-null    object
 10  barang_bukti         492 non-null    object
 11  dakwaan              492 non-null    object
dtypes: int64(1), object(11)
memory usage: 46.3+ KB


# Data Understanding

In [12]:
df.head(5)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,51/Pid.B/2024/PN Amp,Pencurian,"Jumat, 09 Agu. 2024",53 Hari,[I KOMANG ARIMBAWA Alias LELANG],[Ida Ayu Putu Widhiantini],"[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",5,[I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara...,M E N G A D I L I\nMenyatakan Terdakwa I Koman...,DAKWAAN :\n------ Bahwa ia Terdakwa I KOMANG A...
1,Minutasi,43/Pid.Sus/2024/PN Amp,Narkotika,"Kamis, 01 Agu. 2024",35 Hari,[MOH. SOLEH Alias SOLEH],"[Ardi Putra Dewa Agung, S.H]","[Meilia Christina Mulyaningrum~Hakim Ketua~Ya,...",2,[MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bu...,MENGADILI:\nMenyatakan Terdakwa Moh Soleh alia...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...
2,Minutasi,40/Pid.Sus/2024/PN Amp,Narkotika,"Rabu, 24 Jul. 2024",48 Hari,[SOIMUN Alias WAYAN],"[M. Thoriq Ardiansyah, S.H.]","[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",3,[SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan ...,MENGADILI:\n Menyatakan Terdakwa Soimun alias...,DAKWAAN :\n ------- Bahwa Terdakwa SOIMUN ali...
3,Minutasi,49/Pid.B/2024/PN Amp,Pencurian,"Kamis, 08 Agu. 2024",33 Hari,[I NENGAH SEMIARTA],"[Ardi Putra Dewa Agung, S.H]","[Ayu Putri Cempakasari~Hakim Ketua~Ya, Ni Koma...",2,[I NENGAH SEMIARTA~Pidana Penjara Waktu Terten...,M E N G A D I L I\nMenyatakan Terdakwa I Nenga...,Dakwaan\nBahwa Terdakwa I Nengah Semiarta (yan...
4,Minutasi,45/Pid.B/2024/PN Amp,Pencurian,"Senin, 05 Agu. 2024",31 Hari,[I KETUT KERTA],[Ida Ayu Putu Widhiantini],"[Meilia Christina Mulyaningrum~Hakim Ketua~Ya,...",4,[I KETUT KERTA~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Ketut Kerta ...,DAKWAAN :\n------ Bahwa Terdakwa I KETUT KERTA...


**Dataset Features**
| Fitur | Deskripsi | Tipe Data |
|-----------------------|-----------------------------------------------------------------------|:------------:|
| `status_perkara` | Status dari perkara.| `str` |
| `nomor_perkara` | Nomor identifikasi unik untuk perkara.| `str` |
| `klasifikasi_perkara` | Kategori atau jenis perkara.| `str` |
| `tanggal_pendaftaran` | Tanggal ketika perkara didaftarkan di pengadilan.| `str` |
| `lama_proses` | Durasi waktu yang dibutuhkan untuk memproses perkara.| `str` |
| `terdakwa` | Nama individu yang didakwa dalam perkara tersebut.| `list[str]` |
| `penuntut_umum` | Nama jaksa atau penuntut yang menangani perkara.| `list[str]`  |
| `hakim` | Nama hakim yang memimpin sidang perkara.| `list[str]` |
| `jumlah_saksi` | Total jumlah saksi yang dihadirkan dalam sidang.| `int` |
| `putusan_hukuman` | Putusan atau keputusan yang dijatuhkan oleh hakim.| `list[str]` |
| `barang_bukti` | Amar Putusan yang berisi tentang barang bukti perkara.| `str` |
| `dakwaan`| Uraian mengenai tuduhan atau dakwaan yang diajukan terhadap terdakwa.| `str` |







# Data Preprocessing

## Data Cleaning

### Data Cleaning: Null Values

In [13]:
null_counts = df.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


In [14]:
df_copy = df.copy()

### Data Cleaning: Data Disamarkan

#### Check Data "Disamarkan"

In [15]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 45


In [16]:
print(count_disamarkan)

    status_perkara           nomor_perkara            klasifikasi_perkara  \
19        Minutasi  26/Pid.Sus/2024/PN Amp   Kekerasan Dalam Rumah Tangga   
31        Minutasi  58/Pid.Sus/2023/PN Amp              Perlindungan Anak   
35        Minutasi  51/Pid.Sus/2023/PN Amp              Perlindungan Anak   
37        Minutasi  17/Pid.Sus/2024/PN Amp              Perlindungan Anak   
39        Minutasi  14/Pid.Sus/2024/PN Amp              Perlindungan Anak   
46        Minutasi  45/Pid.Sus/2023/PN Amp              Perlindungan Anak   
78        Minutasi    47/Pid.B/2022/PN Amp  Kejahatan terhadap Kesusilaan   
79        Minutasi    45/Pid.B/2022/PN Amp  Kejahatan terhadap Kesusilaan   
104       Minutasi  27/Pid.Sus/2022/PN Amp              Perlindungan Anak   
105       Minutasi  26/Pid.Sus/2022/PN Amp              Perlindungan Anak   
106       Minutasi  24/Pid.Sus/2022/PN Amp              Perlindungan Anak   
124       Minutasi  37/Pid.Sus/2022/PN Amp              Perlindungan Anak   

#### Drop Data "Disamarkan"

In [17]:
df_copy = df_copy[~df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
df_copy.reset_index(drop=True, inplace=True)

In [18]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]

print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 0


### Data Cleaning: Status Perkara

In [19]:
unique_status_perkara_values = df_copy['status_perkara'].unique()
num_unique_status_perkara = len(unique_status_perkara_values)

print(f"Number of unique values in 'status_perkara': {num_unique_status_perkara}")
print("Unique values:", unique_status_perkara_values)

Number of unique values in 'status_perkara': 1
Unique values: ['Minutasi']


### Data Cleaning: Nomor Perkara

In [20]:
unique_nomor_perkara_values = df_copy['nomor_perkara'].unique()
num_unique_nomor_perkara = len(unique_nomor_perkara_values)

print(f"Jumlah Data: {len(df_copy)}")
print(f"Jumlah Nilai Unique pada Kolom 'nomor_perkara': {num_unique_nomor_perkara}")

Jumlah Data: 447
Jumlah Nilai Unique pada Kolom 'nomor_perkara': 447


### Data Cleaning: Klasifikasi Perkara

In [21]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 29
klasifikasi_perkara
Narkotika                                                                                    135
Pencurian                                                                                     99
Penganiayaan                                                                                  38
Lalu Lintas                                                                                   31
Penggelapan                                                                                   20
Kejahatan Perjudian                                                                           20
Penipuan                                                                                      15
Penadahan, Penerbitan, dan Pencetakan                                                         12
Lain-Lain                                                                                      9
Pemalsuan Surat                                            

### TEMP!!

In [22]:
# df_copy = df_copy[~df_copy["klasifikasi_perkara"].isin(["Lalu Lintas"])]
# df_copy.reset_index(drop=True, inplace=True)

In [23]:
# unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
# num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
# unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

# print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
# print(unique_klasifikasi_counts)

### Data Cleaning: Hakim

#### Check "Hakim" Column Data Type (Expected List)

In [24]:
unique_types = df_copy['hakim'].apply(type).unique()
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


In [25]:
count_str_hakim = df_copy[df_copy["hakim"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'hakim' berupa string: {count_str_hakim}")

Jumlah data dengan 'hakim' berupa string: 0


#### Check Unique Values of Total "Hakim"

In [26]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3 6]


In [27]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 6: 
        print(f"{index} -> {row['hakim']}")

10 -> ['Meilia Christina Mulyaningrum~Hakim Ketua~Tidak', 'Luh Putu Sela Septika~Hakim Anggota~Tidak', 'Putu Mas Ayu Cendana Wangi~Hakim Anggota~Tidak', 'Luh Putu Sela Septika~Hakim Ketua~Ya', 'Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya', 'R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya']
114 -> ['Ayu Putri Cempakasari~Hakim Ketua~Tidak', 'Ni Komang Wijiatmawati~Hakim Anggota~Tidak', 'R Aditayoga Nugraha Bimasakti~Hakim Anggota~Tidak', 'Ayu Putri Cempakasari~Hakim Ketua~Ya', 'Luh Putu Sela Septika~Hakim Anggota~Ya', 'R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya']
134 -> ['RONNY WIDODO~Hakim Ketua~Tidak', 'Cokorda Gde Suryalaksana~Hakim Anggota~Tidak', 'Lia Puji Astuti~Hakim Anggota~Tidak', 'RONNY WIDODO~Hakim Ketua~Ya', 'Cokorda Gde Suryalaksana~Hakim Anggota~Ya', 'Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya']
163 -> ['I WAYAN SUARTA~Hakim Ketua~Tidak', 'I Gusti Ayu Kharina Yuli Astiti~Hakim Anggota~Tidak', 'Luh Putu Sela Septika~Hakim Anggota~Tidak', 'I WAYAN SUARTA~Hakim Ketua~Ya',

In [28]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim 

#### Remove Hakim Tidak Aktif

In [29]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [entry for entry in x if '~Tidak' not in entry])

In [30]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim Anggota~Ya

Meilia Christina Mulyaningrum~Hakim Ketua~Ya
Luh Putu Sela Septika~Hakim Anggota~Ya
Putu Mas Ayu Cendana Wangi~Hakim Anggota~Ya

Ayu Putri Cempakasari~Hakim Ketua~Ya
Ni Komang Wijiatmawati~Hakim Anggota~Ya
R Aditayoga Nugraha Bimasakti~Hakim 

#### Check Unique Values after Removing "Hakim Tidak Aktif"

In [31]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3]


#### Remove Hakim Anggota

In [32]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [value for value in x if '~Hakim Anggota' not in value])

In [33]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Luh Putu Sela Septika~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Hakim Ketua~Ya']
['Ayu Putri Cempakasari~Hakim Ketua~Ya']
['Meilia Christina Mulyaningrum~Ha

#### Check Unique Values after Removing "Hakim Anggota" (Expected 1)

In [34]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[1]


#### Extract Hakim Name

In [35]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(
    lambda x: [value.split('~Hakim')[0].strip() for value in x]
)

In [36]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['Luh Putu Sela Septika']
['Meilia Christina Mulyaningrum']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Meilia Christina Mulyaningrum']
['Meilia Christina Mulyaningrum']
['Meilia Christina Mulyaningrum']
['Ayu Putri Cempakasari']
['Putu Mas Ayu Cendana Wangi']
['Ayu Putri Cempakasari']
['Ayu Putri Cempakasari']
['R Aditayoga Nugraha Bimasakti']
['Luh Putu Sela Septika']
['Ayu Putri Cempakasari']
['RONNY WIDODO']
['Ayu Putri Cempakasari']
['Meilia Christina Mulya

#### Convert List to String

In [37]:
df_copy['hakim'] = df_copy['hakim'].apply(lambda x: str(x[0]) if isinstance(x, list) and len(x) > 0 else str(x))

In [38]:
unique_types = df_copy['hakim'].apply(type).unique()
print(unique_types)

[<class 'str'>]


In [39]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Luh Putu Sela Septika
Meilia Christina Mulyaningrum
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Meilia Christina Mulyaningrum
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Putu Mas Ayu Cendana Wangi
Ayu Putri Cempakasari
Ayu Putri Cempakasari
R Aditayoga Nugraha Bimasakti
Luh Putu Sela Septika
Ayu Putri Cempakasari
RONNY WIDODO
Ayu Putri Cempakasari
Meilia Christina Mulyaningrum
Ayu Putri Cempakasari
Luh Putu Sela Septika
Ayu Putri Cempakasari
Ayu Putri Cempakasari
Ayu Putri Cempakasari
RONNY WIDODO
Cokorda

### Data Cleaning: Barang Bukti

#### RegEx Testing on 1 Data

In [40]:
text = df_copy.iloc[3]['barang_bukti']
print(text)

M E N G A D I L I
Menyatakan Terdakwa I Nengah Semiarta telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana ”Pencurian dalam keadaan memberatkan” sebagaimana dakwaan Tunggal Penuntut Umum;
Menjatuhkan pidana terhadap Terdakwa oleh karena itu dengan pidana penjara  selama 2 (dua) tahun;
Menetapkan barang bukti berupa:
satu lembar STNK dari satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 5286 TV, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, di STNK atas nama pemilik I Nengah Suparta;
 satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 4518 SO, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin: JFH1E-1200583, dengan pemilik yang tertera di STNK atas nama I Nengah Suparta;
satu kunci kontak dari satu unit sepeda motor merk Honda Vario F1 tersebut di atas;
Dikembalikan kepada sdri. Ni Luh Candrawati;
Membebankan kepada Terdakwa untuk membayar

##### Remove Unwanted Symbol and Char

In [41]:
cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text)
cleaned_text = re.sub(r'-{2,}', '-', cleaned_text)

In [42]:
print(cleaned_text)

M E N G A D I L I
Menyatakan Terdakwa I Nengah Semiarta telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Pencurian dalam keadaan memberatkan sebagaimana dakwaan Tunggal Penuntut Umum
Menjatuhkan pidana terhadap Terdakwa oleh karena itu dengan pidana penjara  selama 2 (dua) tahun
Menetapkan barang bukti berupa
satu lembar STNK dari satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 5286 TV, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, di STNK atas nama pemilik I Nengah Suparta
 satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 4518 SO, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, dengan pemilik yang tertera di STNK atas nama I Nengah Suparta
satu kunci kontak dari satu unit sepeda motor merk Honda Vario F1 tersebut di atas
Dikembalikan kepada sdri. Ni Luh Candrawati
Membebankan kepada Terdakwa untuk membayar biaya per

##### Remove Multiple Whitespaces

In [43]:
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [44]:
print(cleaned_text)

M E N G A D I L I
Menyatakan Terdakwa I Nengah Semiarta telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Pencurian dalam keadaan memberatkan sebagaimana dakwaan Tunggal Penuntut Umum
Menjatuhkan pidana terhadap Terdakwa oleh karena itu dengan pidana penjara selama 2 (dua) tahun
Menetapkan barang bukti berupa
satu lembar STNK dari satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 5286 TV, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, di STNK atas nama pemilik I Nengah Suparta
 satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 4518 SO, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, dengan pemilik yang tertera di STNK atas nama I Nengah Suparta
satu kunci kontak dari satu unit sepeda motor merk Honda Vario F1 tersebut di atas
Dikembalikan kepada sdri. Ni Luh Candrawati
Membebankan kepada Terdakwa untuk membayar biaya perk

##### Extract Only Barang Bukti

In [45]:
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n\s*[A-Z]|$)"
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n[A-Z])"
pattern = r"menetapkan barang bukti berupa\s*(.*)"

match = re.search(pattern, cleaned_text, re.DOTALL | re.IGNORECASE)

if match:
    print(match.group().strip())
    
else:
    print("Tidak ada kecocokan ditemukan.")

Menetapkan barang bukti berupa
satu lembar STNK dari satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 5286 TV, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, di STNK atas nama pemilik I Nengah Suparta
 satu unit sepeda motor merk Honda Vario F1, nomor registrasi DK 4518 SO, warna putih, tipe ATl1l21B01 A/T, tahun 2014, nomor rangka MH1JFH11XEK200902, nomor mesin JFH1E-1200583, dengan pemilik yang tertera di STNK atas nama I Nengah Suparta
satu kunci kontak dari satu unit sepeda motor merk Honda Vario F1 tersebut di atas
Dikembalikan kepada sdri. Ni Luh Candrawati
Membebankan kepada Terdakwa untuk membayar biaya perkara sejumlah Rp. 2.500,00 (dua ribu lima ratus rupiah)


#### RegEx on Entire DataFrame

##### Extract Barang Bukti

In [46]:
# patterns = [
#     r"barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti,\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang-bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)"
# ]

patterns = [
    r"barang\s*bukti\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti,\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti\s*:?\s*(.*)",
    r"barang-bukti\s*:?\s*(.*)"
]

rows_to_drop = []

for index, row in df_copy.iterrows():
    text_barang_bukti = row['barang_bukti']
    
    cleaned_barang_bukti = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text_barang_bukti)
    cleaned_barang_bukti = re.sub(r'-{2,}', '-', cleaned_barang_bukti)
    cleaned_barang_bukti = re.sub(r' +', ' ', cleaned_barang_bukti)
    
    match_found = False

    for pattern in patterns:
        match = re.search(pattern, cleaned_barang_bukti, re.DOTALL | re.IGNORECASE)

        if match:
            df_copy.loc[index, 'cleaned_barang_bukti'] = match.group(1).strip().replace('\n', ' ')
            match_found = True
            break

    
    if not match_found:
        print(f"{index}. Barang Bukti Tidak Ditemukan pada {row['nomor_perkara']}")
        rows_to_drop.append(index) 
        # print(cleaned_barang_bukti)



36. Barang Bukti Tidak Ditemukan pada 1/Pid.B/2024/PN Amp
213. Barang Bukti Tidak Ditemukan pada 72/Pid.B/2020/PN Amp
228. Barang Bukti Tidak Ditemukan pada 71/Pid.B/2020/PN Amp
274. Barang Bukti Tidak Ditemukan pada 25/Pid.B/2019/PN Amp
276. Barang Bukti Tidak Ditemukan pada 21/Pid.B/2019/PN Amp
306. Barang Bukti Tidak Ditemukan pada 17/Pid.B/2020/PN Amp
338. Barang Bukti Tidak Ditemukan pada 67/Pid.B/2018/PN Amp
350. Barang Bukti Tidak Ditemukan pada 24/Pid.B/2018/PN Amp
406. Barang Bukti Tidak Ditemukan pada 4/Pid.B/2020/PN Amp


In [47]:
df_copy.iloc[36]['barang_bukti']

'MENGADILI:\nMenyatakan Terdakwa Syamsuri tersebut di atas, terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana penipuan sebagaimana dalam dakwaan alternatif kesatu;\nMenjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama 9 (sembilan) bulan;\nMenetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan;\nMenetapkan Terdakwa tetap ditahan;\nMembebankan kepada Terdakwa membayar biaya perkara sejumlah  Rp2.500,00 (dua ribu lima ratus rupiah);'

##### Get Case with No Barang Bukti

In [48]:
true_values = df_copy.isna()
df_fail_barang_bukti = df_copy[true_values['cleaned_barang_bukti']]

In [49]:
df_fail_barang_bukti.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
36,Minutasi,1/Pid.B/2024/PN Amp,Penipuan,"Kamis, 11 Jan. 2024",32 Hari,[SYAMSURI],"[Aditya Toh Prabowo, Aditya Toh Prabowo]",Luh Putu Sela Septika,2,[SYAMSURI~Pidana Penjara Waktu Tertentu (9 Bul...,MENGADILI:\nMenyatakan Terdakwa Syamsuri terse...,KESATU :\nBahwa Terdakwa SYAMSURI pada hari Mi...,NaN
213,Minutasi,72/Pid.B/2020/PN Amp,Penganiayaan,"Selasa, 06 Okt. 2020",36 Hari,[I NYOMAN PUTU ARIAWAN Alias PUTU DIKA],"[I PUTU ERRYC SUNAS ARINTAMA,SH]",Cokorda Gde Suryalaksana,4,[I NYOMAN PUTU ARIAWAN Alias PUTU DIKA~Pidana ...,M E N G A D I L I :\nMenyatakan Terdakw...,Bahwa ia terdakwa I NYOMAN PUTU ARIAWAN Alias ...,NaN
228,Minutasi,71/Pid.B/2020/PN Amp,Penghinaan,"Selasa, 06 Okt. 2020",72 Hari,[I WAYAN MERTA Alias CECEP],"[I PUTU ERRYC SUNAS ARINTAMA,SH]",Ni Made Kushandari,6,[I WAYAN MERTA Alias CECEP~Pidana Penjara Wakt...,MENGADILI:\nMenyatakan Terdakwa I Wayan Merta ...,Bahwa ia terdakwa I WAYAN MERTA Alias CECEP ha...,NaN
274,Minutasi,25/Pid.B/2019/PN Amp,Penganiayaan,"Selasa, 26 Mar. 2019",28 Hari,[I NENGAH SUKARSANA Als. LOSOG],"[I PUTU ERRYC SUNAS ARINTAMA,SH]",Putu Ayu Sudariasih,6,[I NENGAH SUKARSANA Als. LOSOG~Pidana Kurungan...,M E N G A D I L I :\n1. Menyatakan Terdakwa I...,Bahwa ia terdakwa I NENGAH SUKARSANA Alias LOS...,NaN
276,Minutasi,21/Pid.B/2019/PN Amp,Penganiayaan,"Senin, 11 Mar. 2019",35 Hari,[I NENGAH MERTHA],"[NI KADEK DRIPTAYANTI,SH]",Putu Ayu Sudariasih,7,[I NENGAH MERTHA~Pidana Penjara Waktu Tertentu...,M E N G A D I L I : \nMenyatakan Terdakwa I NE...,BahwaTerdakwaI NENGAH MERTHApadahari Selasatan...,NaN


In [50]:
df_fail_barang_bukti.to_csv(f'../Data/Misc/{database_name}_kasus tanpa barang bukti.csv')

##### Drop Perkara Tanpa Barang Bukti

In [51]:
df_copy.drop(rows_to_drop, inplace=True)
print(f"rows dropped: {rows_to_drop}")
print(f"rows dropped: {len(rows_to_drop)}")

rows dropped: [36, 213, 228, 274, 276, 306, 338, 350, 406]
rows dropped: 9


In [52]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
0,Minutasi,51/Pid.B/2024/PN Amp,Pencurian,"Jumat, 09 Agu. 2024",53 Hari,[I KOMANG ARIMBAWA Alias LELANG],[Ida Ayu Putu Widhiantini],Ayu Putri Cempakasari,5,[I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara...,M E N G A D I L I\nMenyatakan Terdakwa I Koman...,DAKWAAN :\n------ Bahwa ia Terdakwa I KOMANG A...,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...
1,Minutasi,43/Pid.Sus/2024/PN Amp,Narkotika,"Kamis, 01 Agu. 2024",35 Hari,[MOH. SOLEH Alias SOLEH],"[Ardi Putra Dewa Agung, S.H]",Meilia Christina Mulyaningrum,2,[MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bu...,MENGADILI:\nMenyatakan Terdakwa Moh Soleh alia...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...,4 (empat) paket narkotika jenis shabu dengan b...
2,Minutasi,40/Pid.Sus/2024/PN Amp,Narkotika,"Rabu, 24 Jul. 2024",48 Hari,[SOIMUN Alias WAYAN],"[M. Thoriq Ardiansyah, S.H.]",Ayu Putri Cempakasari,3,[SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan ...,MENGADILI:\n Menyatakan Terdakwa Soimun alias...,DAKWAAN :\n ------- Bahwa Terdakwa SOIMUN ali...,2 paket klip plastic berisi serbuk kristal ben...
3,Minutasi,49/Pid.B/2024/PN Amp,Pencurian,"Kamis, 08 Agu. 2024",33 Hari,[I NENGAH SEMIARTA],"[Ardi Putra Dewa Agung, S.H]",Ayu Putri Cempakasari,2,[I NENGAH SEMIARTA~Pidana Penjara Waktu Terten...,M E N G A D I L I\nMenyatakan Terdakwa I Nenga...,Dakwaan\nBahwa Terdakwa I Nengah Semiarta (yan...,satu lembar STNK dari satu unit sepeda motor m...
4,Minutasi,45/Pid.B/2024/PN Amp,Pencurian,"Senin, 05 Agu. 2024",31 Hari,[I KETUT KERTA],[Ida Ayu Putu Widhiantini],Meilia Christina Mulyaningrum,4,[I KETUT KERTA~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Ketut Kerta ...,DAKWAAN :\n------ Bahwa Terdakwa I KETUT KERTA...,1 (satu) buah toples plastik bening dengan tut...


##### Check NaN Values

In [53]:
null_counts = df_copy.isna().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
dtype: int64


### Data Cleaning: Dakwaan

#### Test Cleaning on 1 Data

In [54]:
dakwaan = df_copy.iloc[33]['dakwaan']
print(dakwaan)

KEJAKSAAN NEGERI KARANGASEM
P - 29
“Demi Keadilan dan Kebenaran Berdasarkan Ketuhanan Yang Maha Esa”
    SURAT DAKWAAN
Reg. Perk. No. : PDM 43/KR.ASEM/09/2023
  Identitas Terdakwa
Nama terdakwa
:
I Wayan Kariana Alias Wayan Ompong.
Nomor identitas
:
5107011504890002.
Tempat lahir
:
Buktabya.
Umur / tgl lahir
:
37 Tahun / 28 April 1986.
Jenis kelamin
:
Laki-laki.
Kebangsaan / kewarganegaraan
:
Indonesia.
Tempat tinggal
:
Br. Dinas Batusesa, Desa/ Kelurahan Menanga, Kecamatan Rendang, Kabupaten Karangasem, Provinsi Bali.
Agama
:
Hindu.
Pekerjaan
:
Wiraswasta.
Pendidikan
:
SMP (berijazah).
  II.    Penahanan (Rutan)
1.
Oleh Penyidik
:
Sejak tanggal 16 Juni 2023 sampai dengan tanggal 5 Juli 2023;
2.
Perpanjangan oleh Kepala Kejaksaan Negeri Karangasem
:
Sejak tanggal 6 Juli 2023 sampai dengan tanggal 14 Agustus 2023;
3.
Perpanjangan I oleh Ketua Pengadilan Negeri Amlapura
:
Sejak tanggal 15 Agustus 2023 sampai dengan tanggal 13 September 2023;
4.
Perpanjangan II oleh Ketua Pengadilan Neger

##### Clean Unwanted Char and Symbols

In [55]:
# cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n]+', dakwaan)
# cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
# print(cleaned_dakwaan)

cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', dakwaan)

cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)

print(cleaned_dakwaan)

KEJAKSAAN NEGERI KARANGASEM
P - 29
Demi Keadilan dan Kebenaran Berdasarkan Ketuhanan Yang Maha Esa
 SURAT DAKWAAN
Reg. Perk. No. PDM 43/KR.ASEM/09/2023
 Identitas Terdakwa
Nama terdakwa

I Wayan Kariana Alias Wayan Ompong.
Nomor identitas

5107011504890002.
Tempat lahir

Buktabya.
Umur / tgl lahir

37 Tahun / 28 April 1986.
Jenis kelamin

Laki-laki.
Kebangsaan / kewarganegaraan

Indonesia.
Tempat tinggal

Br. Dinas Batusesa, Desa/ Kelurahan Menanga, Kecamatan Rendang, Kabupaten Karangasem, Provinsi Bali.
Agama

Hindu.
Pekerjaan

Wiraswasta.
Pendidikan

SMP (berijazah).
 II. Penahanan (Rutan)
1.
Oleh Penyidik

Sejak tanggal 16 Juni 2023 sampai dengan tanggal 5 Juli 2023
2.
Perpanjangan oleh Kepala Kejaksaan Negeri Karangasem

Sejak tanggal 6 Juli 2023 sampai dengan tanggal 14 Agustus 2023
3.
Perpanjangan I oleh Ketua Pengadilan Negeri Amlapura

Sejak tanggal 15 Agustus 2023 sampai dengan tanggal 13 September 2023
4.
Perpanjangan II oleh Ketua Pengadilan Negeri Amlapura

Sejak tanggal 14

##### Remove Header

In [56]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

Dakwaan
Kesatu
Bahwa Terdakwa I Wayan Kariana Alias Wayan Ompong (yang selanjutnya disebut terdakwa) pada hari Minggu tanggal 11 Juni 2023 sekira pukul 18.30 WITA atau pada waktu lain pada tahun 2023 bertempat di sebuah gudang parkir truk, tepatnya di Banjar Dinas Geria, Desa Rendang, Kecamatan Rendang, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, telah melakukan tindak pidana tanpa hak atau melawan hukum memiliki, menyimpan, menguasai, atau menyediakan Narkotika Golongan I, perbuatan terdakwa dilakukan dengan cara sebagai berikut
Bahwa awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi I Nyoman Budi Adnyana yang sebelumnya telah mendapat informasi ada aktivitas mencurigakan di sebuah gudang parkir truk, tepatnya di Banjar Dinas Geria, Desa Rendang pada sekira pukul 18.30 Wita mencegat terdakwa yang b

##### Remove Footer

In [57]:
pattern = r"^(.*?)\s+Amlapura,\s+\d{1,2}\s+\w+\s+\d{4}"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

Dakwaan
Kesatu
Bahwa Terdakwa I Wayan Kariana Alias Wayan Ompong (yang selanjutnya disebut terdakwa) pada hari Minggu tanggal 11 Juni 2023 sekira pukul 18.30 WITA atau pada waktu lain pada tahun 2023 bertempat di sebuah gudang parkir truk, tepatnya di Banjar Dinas Geria, Desa Rendang, Kecamatan Rendang, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, telah melakukan tindak pidana tanpa hak atau melawan hukum memiliki, menyimpan, menguasai, atau menyediakan Narkotika Golongan I, perbuatan terdakwa dilakukan dengan cara sebagai berikut
Bahwa awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi I Nyoman Budi Adnyana yang sebelumnya telah mendapat informasi ada aktivitas mencurigakan di sebuah gudang parkir truk, tepatnya di Banjar Dinas Geria, Desa Rendang pada sekira pukul 18.30 Wita mencegat terdakwa yang b

#### Cleaning on Entire DataFrame

##### Clean Unwanted Char & Symbols

In [58]:
def clean_dakwaan(text):
    cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', text)
    cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
    
    cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
    cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)
    
    return cleaned_dakwaan

In [59]:
df_copy['cleaned_dakwaan'] = df_copy['dakwaan'].apply(clean_dakwaan)

In [60]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,51/Pid.B/2024/PN Amp,Pencurian,"Jumat, 09 Agu. 2024",53 Hari,[I KOMANG ARIMBAWA Alias LELANG],[Ida Ayu Putu Widhiantini],Ayu Putri Cempakasari,5,[I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara...,M E N G A D I L I\nMenyatakan Terdakwa I Koman...,DAKWAAN :\n------ Bahwa ia Terdakwa I KOMANG A...,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...,DAKWAAN \n- Bahwa ia Terdakwa I KOMANG ARIMBAW...
1,Minutasi,43/Pid.Sus/2024/PN Amp,Narkotika,"Kamis, 01 Agu. 2024",35 Hari,[MOH. SOLEH Alias SOLEH],"[Ardi Putra Dewa Agung, S.H]",Meilia Christina Mulyaningrum,2,[MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bu...,MENGADILI:\nMenyatakan Terdakwa Moh Soleh alia...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...,4 (empat) paket narkotika jenis shabu dengan b...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...
2,Minutasi,40/Pid.Sus/2024/PN Amp,Narkotika,"Rabu, 24 Jul. 2024",48 Hari,[SOIMUN Alias WAYAN],"[M. Thoriq Ardiansyah, S.H.]",Ayu Putri Cempakasari,3,[SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan ...,MENGADILI:\n Menyatakan Terdakwa Soimun alias...,DAKWAAN :\n ------- Bahwa Terdakwa SOIMUN ali...,2 paket klip plastic berisi serbuk kristal ben...,DAKWAAN \n - Bahwa Terdakwa SOIMUN alias WAYAN...
3,Minutasi,49/Pid.B/2024/PN Amp,Pencurian,"Kamis, 08 Agu. 2024",33 Hari,[I NENGAH SEMIARTA],"[Ardi Putra Dewa Agung, S.H]",Ayu Putri Cempakasari,2,[I NENGAH SEMIARTA~Pidana Penjara Waktu Terten...,M E N G A D I L I\nMenyatakan Terdakwa I Nenga...,Dakwaan\nBahwa Terdakwa I Nengah Semiarta (yan...,satu lembar STNK dari satu unit sepeda motor m...,Dakwaan\nBahwa Terdakwa I Nengah Semiarta (yan...
4,Minutasi,45/Pid.B/2024/PN Amp,Pencurian,"Senin, 05 Agu. 2024",31 Hari,[I KETUT KERTA],[Ida Ayu Putu Widhiantini],Meilia Christina Mulyaningrum,4,[I KETUT KERTA~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Ketut Kerta ...,DAKWAAN :\n------ Bahwa Terdakwa I KETUT KERTA...,1 (satu) buah toples plastik bening dengan tut...,DAKWAAN \n- Bahwa Terdakwa I KETUT KERTA (sela...


##### Remove Header

In [61]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

    if matches: 
        last_match = matches[-1]
        df_copy.loc[index, "cleaned_dakwaan"] = last_match.strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 30
raw dakwaan copied at 31
raw dakwaan copied at 38
raw dakwaan copied at 50
raw dakwaan copied at 52
raw dakwaan copied at 55
raw dakwaan copied at 58
raw dakwaan copied at 67
raw dakwaan copied at 68
raw dakwaan copied at 69
raw dakwaan copied at 70
raw dakwaan copied at 71
raw dakwaan copied at 72
raw dakwaan copied at 73
raw dakwaan copied at 74
raw dakwaan copied at 75
raw dakwaan copied at 76
raw dakwaan copied at 77
raw dakwaan copied at 78
raw dakwaan copied at 79
raw dakwaan copied at 80
raw dakwaan copied at 81
raw dakwaan copied at 82
raw dakwaan copied at 92
raw dakwaan copied at 93
raw dakwaan copied at 94
raw dakwaan copied at 95
raw dakwaan copied at 96
raw dakwaan copied at 97
raw dakwaan copied at 98
raw dakwaan copied at 99
raw dakwaan copied at 105
raw dakwaan copied at 106
raw dakwaan copied at 107
raw dakwaan copied at 108
raw dakwaan copied at 109
raw dakwaan copied at 110
raw dakwaan copied at 111
raw dakwaan copied at 112
raw dakwaan copie

In [62]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan
Kesatu
Bahwa Terdakwa Moh Soleh Alias Soleh (yang selanjutnya disebut terdakwa) pada hari Senin tanggal 13 Mei 2024 sekira pukul 00.10 Wita atau pada waktu lain pada tahun 2024 bertempat di pinggir jalan raya depan lapangan tenis tepatnya di Jalan Untung Surapati, Kelurahan Subagan, Kecamatan Karangasem, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, melakukan tindak pidana tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, perbuatan terdakwa tersebut dilakukan dengan cara sebagai berikut
Awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi Ida Bagus Yogi Pramana Putra yang sekira pukul 00.10 Wita melihat sepeda motor merek Honda Beat berwarna deluxe blue dengan plat DK 3281 AEG terpar

##### Remove Footer

In [63]:
pattern = r"^(.*?)\s+Amlapura,\s+\d{1,2}\s+\w+\s+\d{4}"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.match(pattern, text_dakwaan, re.DOTALL)

    if matches: 
        df_copy.loc[index, "cleaned_dakwaan"] = matches.group(1).strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")

raw dakwaan copied at 0
raw dakwaan copied at 1
raw dakwaan copied at 2
raw dakwaan copied at 3
raw dakwaan copied at 4
raw dakwaan copied at 5
raw dakwaan copied at 6
raw dakwaan copied at 7
raw dakwaan copied at 8
raw dakwaan copied at 9
raw dakwaan copied at 10
raw dakwaan copied at 11
raw dakwaan copied at 12
raw dakwaan copied at 13
raw dakwaan copied at 14
raw dakwaan copied at 15
raw dakwaan copied at 16
raw dakwaan copied at 17
raw dakwaan copied at 18
raw dakwaan copied at 19
raw dakwaan copied at 20
raw dakwaan copied at 21
raw dakwaan copied at 22
raw dakwaan copied at 23
raw dakwaan copied at 24
raw dakwaan copied at 25
raw dakwaan copied at 26
raw dakwaan copied at 27
raw dakwaan copied at 28
raw dakwaan copied at 29
raw dakwaan copied at 30
raw dakwaan copied at 31
raw dakwaan copied at 32
raw dakwaan copied at 34
raw dakwaan copied at 35
raw dakwaan copied at 37
raw dakwaan copied at 38
raw dakwaan copied at 42
raw dakwaan copied at 43
raw dakwaan copied at 44
raw dakwaa

In [64]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan
Kesatu
Bahwa Terdakwa Moh Soleh Alias Soleh (yang selanjutnya disebut terdakwa) pada hari Senin tanggal 13 Mei 2024 sekira pukul 00.10 Wita atau pada waktu lain pada tahun 2024 bertempat di pinggir jalan raya depan lapangan tenis tepatnya di Jalan Untung Surapati, Kelurahan Subagan, Kecamatan Karangasem, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, melakukan tindak pidana tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, perbuatan terdakwa tersebut dilakukan dengan cara sebagai berikut
Awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi Ida Bagus Yogi Pramana Putra yang sekira pukul 00.10 Wita melihat sepeda motor merek Honda Beat berwarna deluxe blue dengan plat DK 3281 AEG terpar

In [65]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('\n', '. ', regex=False)

In [66]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alias Soleh (yang selanjutnya disebut terdakwa) pada hari Senin tanggal 13 Mei 2024 sekira pukul 00.10 Wita atau pada waktu lain pada tahun 2024 bertempat di pinggir jalan raya depan lapangan tenis tepatnya di Jalan Untung Surapati, Kelurahan Subagan, Kecamatan Karangasem, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, melakukan tindak pidana tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, perbuatan terdakwa tersebut dilakukan dengan cara sebagai berikut. Awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi Ida Bagus Yogi Pramana Putra yang sekira pukul 00.10 Wita melihat sepeda motor merek Honda Beat berwarna deluxe blue dengan plat DK 3281 AEG ter

In [67]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('  ', ' ', regex=False)

In [68]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alias Soleh (yang selanjutnya disebut terdakwa) pada hari Senin tanggal 13 Mei 2024 sekira pukul 00.10 Wita atau pada waktu lain pada tahun 2024 bertempat di pinggir jalan raya depan lapangan tenis tepatnya di Jalan Untung Surapati, Kelurahan Subagan, Kecamatan Karangasem, Kabupaten Karangasem atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Amlapura yang berwenang memeriksa dan mengadili perkara ini, melakukan tindak pidana tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I, perbuatan terdakwa tersebut dilakukan dengan cara sebagai berikut. Awalnya anggota Kepolisian Resor Karangasem diantaranya Saksi I Made Agus Arta Dwicaksana dan Saksi Ida Bagus Yogi Pramana Putra yang sekira pukul 00.10 Wita melihat sepeda motor merek Honda Beat berwarna deluxe blue dengan plat DK 3281 AEG ter

In [69]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace(' .', '.', regex=False)

In [70]:
df_copy.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

### Save to CSV - STAGE 1 PREPROCESSING

In [71]:
df_copy.to_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv', index=False)

## Data Transformation

#### Data Transformation: Import Data

In [72]:
df_clean = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv')
df_clean.head(3)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,51/Pid.B/2024/PN Amp,Pencurian,"Jumat, 09 Agu. 2024",53 Hari,['I KOMANG ARIMBAWA Alias LELANG'],['Ida Ayu Putu Widhiantini'],Ayu Putri Cempakasari,5,['I KOMANG ARIMBAWA Alias LELANG~Pidana Penjar...,M E N G A D I L I\nMenyatakan Terdakwa I Koman...,DAKWAAN :\n------ Bahwa ia Terdakwa I KOMANG A...,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...,DAKWAAN. - Bahwa ia Terdakwa I KOMANG ARIMBAWA...
1,Minutasi,43/Pid.Sus/2024/PN Amp,Narkotika,"Kamis, 01 Agu. 2024",35 Hari,['MOH. SOLEH Alias SOLEH'],"['Ardi Putra Dewa Agung, S.H']",Meilia Christina Mulyaningrum,2,['MOH. SOLEH Alias SOLEH~Subsider Penjara (8 B...,MENGADILI:\nMenyatakan Terdakwa Moh Soleh alia...,Dakwaan\nKesatu\nBahwa Terdakwa Moh Soleh Alia...,4 (empat) paket narkotika jenis shabu dengan b...,Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alia...
2,Minutasi,40/Pid.Sus/2024/PN Amp,Narkotika,"Rabu, 24 Jul. 2024",48 Hari,['SOIMUN Alias WAYAN'],"['M. Thoriq Ardiansyah, S.H.']",Ayu Putri Cempakasari,3,['SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan...,MENGADILI:\n Menyatakan Terdakwa Soimun alias...,DAKWAAN :\n ------- Bahwa Terdakwa SOIMUN ali...,2 paket klip plastic berisi serbuk kristal ben...,"DAKWAAN. - Bahwa Terdakwa SOIMUN alias WAYAN, ..."


In [73]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 2
cleaned_barang_bukti    0
cleaned_dakwaan         2
dtype: int64

In [5]:
null_rows = df_clean[df_clean.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

Rows with null values:
   status_perkara           nomor_perkara klasifikasi_perkara  \
31       Minutasi  52/Pid.Sus/2023/PN Amp           Narkotika   
51       Minutasi  41/Pid.Sus/2023/PN Amp           Narkotika   

     tanggal_pendaftaran lama_proses                              terdakwa  \
31  Selasa, 24 Okt. 2023     58 Hari      ['I PUTU AGUS PUTRA Alias AGUS']   
51   Jumat, 08 Sep. 2023     59 Hari  ['I WAYAN GEDE ARIADI alias KRODOK']   

                     penuntut_umum                  hakim  jumlah_saksi  \
31  ['M. Thoriq Ardiansyah, S.H.']  Ayu Putri Cempakasari             2   
51  ['M. Thoriq Ardiansyah, S.H.']           RONNY WIDODO             4   

                                      putusan_hukuman  \
31  ['I PUTU AGUS PUTRA Alias AGUS~Subsider Penjar...   
51  ['I WAYAN GEDE ARIADI alias KRODOK~Pidana Penj...   

                                         barang_bukti dakwaan  \
31  MENGADILI:\n  Menyatakan Terdakwa I Putu Agus ...     NaN   
51  MENGADILI:\nMe

#### Data Transformation: Drop Null Values

In [74]:
df_clean.dropna(inplace=True)

In [75]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

#### Data Transformation: Drop Unused Columns

In [76]:
selected_columns = ['nomor_perkara', 
                    'klasifikasi_perkara', 
                    'terdakwa', 
                    'penuntut_umum', 
                    'hakim', 
                    'jumlah_saksi', 
                    'putusan_hukuman', 
                    'cleaned_barang_bukti', 
                    'cleaned_dakwaan']

df_clean = df_clean[selected_columns].copy()

df_clean.head(3)

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,51/Pid.B/2024/PN Amp,Pencurian,['I KOMANG ARIMBAWA Alias LELANG'],['Ida Ayu Putu Widhiantini'],Ayu Putri Cempakasari,5,['I KOMANG ARIMBAWA Alias LELANG~Pidana Penjar...,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...,DAKWAAN. - Bahwa ia Terdakwa I KOMANG ARIMBAWA...
1,43/Pid.Sus/2024/PN Amp,Narkotika,['MOH. SOLEH Alias SOLEH'],"['Ardi Putra Dewa Agung, S.H']",Meilia Christina Mulyaningrum,2,['MOH. SOLEH Alias SOLEH~Subsider Penjara (8 B...,4 (empat) paket narkotika jenis shabu dengan b...,Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alia...
2,40/Pid.Sus/2024/PN Amp,Narkotika,['SOIMUN Alias WAYAN'],"['M. Thoriq Ardiansyah, S.H.']",Ayu Putri Cempakasari,3,['SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan...,2 paket klip plastic berisi serbuk kristal ben...,"DAKWAAN. - Bahwa Terdakwa SOIMUN alias WAYAN, ..."


### Data Transformation: Extract Multiple Terdakwa & Putusan Hukuman to each Row 

#### Check Data Type (Expected List)

In [77]:
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa string: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa string: {count_str_terdakwa}")

Jumlah Data: 436
Jumlah data dengan 'terdakwa' berupa string: 436
Jumlah data dengan 'putusan_hukuman' berupa string: 436


#### Convert To List

In [78]:
def clean_and_convert_to_list(terdakwa):
    if isinstance(terdakwa, list):
        return terdakwa
    
    if isinstance(terdakwa, str):
        cleaned_str = re.sub(r'\s+', ' ', terdakwa.strip())
        try:

            result_list = ast.literal_eval(cleaned_str)
        except (ValueError, SyntaxError):
            result_list = [cleaned_str]

        if isinstance(result_list, str):
            return [result_list]
        return result_list
    return []

df_clean['terdakwa'] = df_clean['terdakwa'].apply(clean_and_convert_to_list)
df_clean['putusan_hukuman'] = df_clean['putusan_hukuman'].apply(clean_and_convert_to_list)

In [79]:
print(df_clean['terdakwa'].apply(type).unique())
print(df_clean['putusan_hukuman'].apply(type).unique())
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa list: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa list: {count_str_terdakwa}")

[<class 'list'>]
[<class 'list'>]
Jumlah Data: 436
Jumlah data dengan 'terdakwa' berupa list: 436
Jumlah data dengan 'putusan_hukuman' berupa list: 436


In [80]:
print(df_clean.loc[18, "putusan_hukuman"])
print(df_clean.loc[18, "terdakwa"])

['I GEDE RISKI ETIKA CANDRA Alias TAPAK~Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )']
['I GEDE RISKI ETIKA CANDRA Alias TAPAK']


#### Check if Jumlah terdakwa tidak sama dengan Jumlah Putusan (More Data Cleaning)

In [81]:
# Check number of elements in each list
df_clean['terdakwa_len'] = df_clean['terdakwa'].apply(len)
df_clean['putusan_len'] = df_clean['putusan_hukuman'].apply(len)

# Display rows where the lengths are not equal
mismatched_rows = df_clean[df_clean['terdakwa_len'] != df_clean['putusan_len']]
print(mismatched_rows[['terdakwa_len', 'putusan_len']].head())


     terdakwa_len  putusan_len
292             1            0
293             1            0
294             1            0
295             1            0
351             1            0


In [82]:
print(mismatched_rows)

              nomor_perkara                klasifikasi_perkara  \
292    47/Pid.B/2017/PN Amp                           Penipuan   
293  34/Pid.Sus/2017/PN Amp                          Narkotika   
294  29/Pid.Sus/2017/PN Amp  Pertambangan Mineral dan Batubara   
295  35/Pid.Sus/2016/PN Amp                          Narkotika   
351    57/Pid.B/2017/PN Amp                          Pencurian   
437    24/Pid.B/2017/PN Amp                           Penipuan   

                                terdakwa  \
292       [NI WAYAN SRIWATI ALIAS TATIK]   
293       [I Kadek Sumandi Als. Dek Tag]   
294  [IR.YUYUN YANWAR YUNUS Alias YUYUN]   
295       [I Kadek Muliana als. Dek Mul]   
351                   [IDA MADE WIYANTA]   
437                   [MUHIBBIN ALS BEN]   

                                         penuntut_umum                hakim  \
292                             ['NUR APRILIYANTO,SH']     I Ketut Kimiarsa   
293                            ['I MADE SANTIAWAN,SH']  Putu Ayu Sudar

#### Copy Only if Terdakwa and Putusan is Same 

In [83]:
df_clean = df_clean[df_clean['terdakwa_len'] == df_clean['putusan_len']]
print(df_clean.shape)

(430, 11)


#### Explode Terdakwa and Putusan

In [84]:
def match_terdakwa_to_putusan(df):
    combined_terdakwa_putusan_list = []
    for index, row in df.iterrows():
        for terdakwa in row['terdakwa']:
            matched = False
            for putusan_hukuman in row['putusan_hukuman']:
                if terdakwa in putusan_hukuman:
                    temp_list = [row['nomor_perkara'], row['klasifikasi_perkara'], row['penuntut_umum'], row['hakim'], row['jumlah_saksi'], row['cleaned_barang_bukti'], row['cleaned_dakwaan'],terdakwa, putusan_hukuman] 
                    combined_terdakwa_putusan_list.append(temp_list)
                    matched = True
                    break
            if not matched:
                print(f'no match {terdakwa} at index {index}')
    df_combined = pd.DataFrame(combined_terdakwa_putusan_list, columns=['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'putusan_hukuman'])

    return df_combined

df_clean_exploded = match_terdakwa_to_putusan(df_clean)
print(df_clean_exploded)

              nomor_perkara  \
0      51/Pid.B/2024/PN Amp   
1    43/Pid.Sus/2024/PN Amp   
2    40/Pid.Sus/2024/PN Amp   
3      49/Pid.B/2024/PN Amp   
4      45/Pid.B/2024/PN Amp   
..                      ...   
520    19/Pid.B/2018/PN Amp   
521    19/Pid.B/2018/PN Amp   
522    19/Pid.B/2018/PN Amp   
523    17/Pid.B/2018/PN Amp   
524   112/Pid.B/2017/PN Amp   

                                   klasifikasi_perkara  \
0                                            Pencurian   
1                                            Narkotika   
2                                            Narkotika   
3                                            Pencurian   
4                                            Pencurian   
..                                                 ...   
520  Kejahatan yang Membahayakan Keamananan Umum Ba...   
521  Kejahatan yang Membahayakan Keamananan Umum Ba...   
522  Kejahatan yang Membahayakan Keamananan Umum Ba...   
523              Penadahan, Penerbitan, dan Pen

In [85]:
df_clean_exploded[['nomor_perkara', 'terdakwa', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,putusan_hukuman
0,51/Pid.B/2024/PN Amp,I KOMANG ARIMBAWA Alias LELANG,I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara ...
1,43/Pid.Sus/2024/PN Amp,MOH. SOLEH Alias SOLEH,MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bul...
2,40/Pid.Sus/2024/PN Amp,SOIMUN Alias WAYAN,SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan )...
3,49/Pid.B/2024/PN Amp,I NENGAH SEMIARTA,I NENGAH SEMIARTA~Pidana Penjara Waktu Tertent...
4,45/Pid.B/2024/PN Amp,I KETUT KERTA,I KETUT KERTA~Pidana Penjara Waktu Tertentu (1...


In [86]:
print(df_clean.shape)
print(df_clean_exploded.shape)
print(df_clean_exploded.isna().sum())

(430, 11)
(525, 9)
nomor_perkara           0
klasifikasi_perkara     0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
cleaned_barang_bukti    0
cleaned_dakwaan         0
terdakwa                0
putusan_hukuman         0
dtype: int64


In [87]:
print(len(df_clean_exploded['terdakwa']))
print(len(df_clean_exploded['putusan_hukuman']))

525
525


In [88]:
for i in range(10):
    print(i, ".", df_clean_exploded.iloc[i]['putusan_hukuman'])

0 . I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1 . MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bulan )
Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )
2 . SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan )
Pidana Denda Rp.800.000.000,00
Pidana Penjara Waktu Tertentu (6 Tahun )
3 . I NENGAH SEMIARTA~Pidana Penjara Waktu Tertentu (2 Tahun )
4 . I KETUT KERTA~Pidana Penjara Waktu Tertentu (11 Bulan )
5 . I GEDE CIDRA~Subsider Kurungan (1 Bulan )
Pidana Denda Rp.5.000.000,00
Pidana Penjara Waktu Tertentu (1 Bulan )
6 . KADEK HENDRA FEBRIANA Alias ADUL~Subsider Penjara (6 Bulan )
Pidana Denda Rp.800.000.000,00
Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
7 . LUTFI BIN FADLAN~Pidana Penjara Waktu Tertentu (6 Bulan )
8 . I KOMANG ARI HANDIKA Alias KOMING~Subsider Penjara (6 Bulan )
Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )
9 . I KETUT SUDARTA Alias MONJONG~Subsider Penjara (6 Bulan )


### Data transformation: Extract Multiple Penuntut Umum

#### Check Data Type (Expected List)

In [89]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'>]


#### Convert to List

In [90]:
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [91]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


#### Check Penuntut Umum > 1

In [92]:
unique_types = df_clean_exploded['penuntut_umum'].apply(len).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [1 2]


In [93]:
list_lengths = df_clean_exploded['penuntut_umum'].apply(len)

unique_lengths = list_lengths.unique()

print(f"Unique lengths of data in the 'penuntut_umum' column: {unique_lengths}")
print("Indices for each unique length:")

for length in unique_lengths:
    indices = list_lengths[list_lengths == length].index
    print(f"Length {length}: Indices - {indices.tolist()}")


Unique lengths of data in the 'penuntut_umum' column: [1 2]
Indices for each unique length:
Length 1: Indices - [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 99, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 179, 180, 181, 182, 183, 185, 189, 190, 191, 193, 194, 195, 196, 198, 199, 200, 201, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [94]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

11 ['Ardi Putra Dewa Agung, S.H', 'Ardi Putra Dewa Agung, S.H']
51 ['M. Thoriq Ardiansyah, S.H.', 'Dewa Gede Angga Pratipta, SH.']
97 ['KADEK ADI PRAMARTA, SH', 'NI KADEK DRIPTAYANTI,SH']
98 ['ASTRID MEIRIKA,SH.', 'ANISA DWILIANA,SH.']
100 ['ARDI PUTRA DEWA AGUNG, S.H.', 'BOBY HERLAMBANG,SH.MH.']
101 ['I DEWA GEDE SEMARA PUTRA, SH', 'ANISA DWILIANA,SH.']
102 ['KADEK WIRA ATMAJA,SH.', 'ARDI PUTRA DEWA AGUNG, S.H.']
143 ['I Dewa Gede Semara Putra', 'Ardi Putra Dewa Agung, S.H']
156 ['ASTRID MEIRIKA,SH.', 'ARDI PUTRA DEWA AGUNG, S.H.']
164 ['BOBY HERLAMBANG,SH.MH.', 'ASTRID MEIRIKA,SH.']
174 ['KADEK WIRA ATMAJA,SH.', 'NI KADEK DRIPTAYANTI,SH']
177 ['KADEK WIRA ATMAJA,SH.', 'ANISA DWILIANA,SH.']
178 ['I DEWA GEDE SEMARA PUTRA, SH', 'BOBY HERLAMBANG,SH.MH.']
184 ['I DEWA GEDE SEMARA PUTRA, SH', 'BOBY HERLAMBANG,SH.MH.']
186 ['I DEWA GEDE SEMARA PUTRA, SH', 'KADEK AYU DYAH UTAMI DEWI, SH.,MH']
187 ['I DEWA GEDE SEMARA PUTRA, SH', 'KADEK AYU DYAH UTAMI DEWI, SH.,MH']
188 ['BOBY HERLAMBANG,SH.

#### Fix Duplicates Penuntut Umum

In [95]:
# df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set([name.lower() for name in x])))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set(x)))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: sorted(x))

In [96]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

51 ['Dewa Gede Angga Pratipta, SH.', 'M. Thoriq Ardiansyah, S.H.']
97 ['KADEK ADI PRAMARTA, SH', 'NI KADEK DRIPTAYANTI,SH']
98 ['ANISA DWILIANA,SH.', 'ASTRID MEIRIKA,SH.']
100 ['ARDI PUTRA DEWA AGUNG, S.H.', 'BOBY HERLAMBANG,SH.MH.']
101 ['ANISA DWILIANA,SH.', 'I DEWA GEDE SEMARA PUTRA, SH']
102 ['ARDI PUTRA DEWA AGUNG, S.H.', 'KADEK WIRA ATMAJA,SH.']
143 ['Ardi Putra Dewa Agung, S.H', 'I Dewa Gede Semara Putra']
156 ['ARDI PUTRA DEWA AGUNG, S.H.', 'ASTRID MEIRIKA,SH.']
164 ['ASTRID MEIRIKA,SH.', 'BOBY HERLAMBANG,SH.MH.']
174 ['KADEK WIRA ATMAJA,SH.', 'NI KADEK DRIPTAYANTI,SH']
177 ['ANISA DWILIANA,SH.', 'KADEK WIRA ATMAJA,SH.']
178 ['BOBY HERLAMBANG,SH.MH.', 'I DEWA GEDE SEMARA PUTRA, SH']
184 ['BOBY HERLAMBANG,SH.MH.', 'I DEWA GEDE SEMARA PUTRA, SH']
186 ['I DEWA GEDE SEMARA PUTRA, SH', 'KADEK AYU DYAH UTAMI DEWI, SH.,MH']
187 ['I DEWA GEDE SEMARA PUTRA, SH', 'KADEK AYU DYAH UTAMI DEWI, SH.,MH']
188 ['ASTRID MEIRIKA,SH.', 'BOBY HERLAMBANG,SH.MH.']
192 ['ASTRID MEIRIKA,SH.', 'I DEWA G

#### Explode DataFrame

In [97]:
df_clean_penuntut_exploded = df_clean_exploded.explode('penuntut_umum', ignore_index=True)

In [98]:
df_clean_penuntut_exploded[['nomor_perkara', 'terdakwa', 'penuntut_umum', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,penuntut_umum,putusan_hukuman
0,51/Pid.B/2024/PN Amp,I KOMANG ARIMBAWA Alias LELANG,Ida Ayu Putu Widhiantini,I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara ...
1,43/Pid.Sus/2024/PN Amp,MOH. SOLEH Alias SOLEH,"Ardi Putra Dewa Agung, S.H",MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bul...
2,40/Pid.Sus/2024/PN Amp,SOIMUN Alias WAYAN,"M. Thoriq Ardiansyah, S.H.",SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan )...
3,49/Pid.B/2024/PN Amp,I NENGAH SEMIARTA,"Ardi Putra Dewa Agung, S.H",I NENGAH SEMIARTA~Pidana Penjara Waktu Tertent...
4,45/Pid.B/2024/PN Amp,I KETUT KERTA,Ida Ayu Putu Widhiantini,I KETUT KERTA~Pidana Penjara Waktu Tertentu (1...


### Data Transformation: Extract Lama Kurungan Penjara

In [99]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                       I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1                MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bulan )\nPidana Denda Rp.1.000.000.000,00\nPidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )
2                              SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan )\nPidana Denda Rp.800.000.000,00\nPidana Penjara Waktu Tertentu (6 Tahun )
3                                                                                            I NENGAH SEMIARTA~Pidana Penjara Waktu Tertentu (2 Tahun )
4                                                                                               I KETUT KERTA~Pidana Penjara Waktu Tertentu (11 Bulan )
5                                     I GEDE CIDRA~Subsider Kurungan (1 Bulan )\nPidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Bulan )
6        KADEK HENDRA FEBRIANA Alias ADUL~Subsider Penjara (6 Bulan )\nPidana Denda Rp.8

#### Remove Terdakwa Name from Putusan Hukuman

In [100]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['putusan_hukuman'].apply(lambda x: x[0].split("~")[1] if isinstance(x, list) and len(x) > 0 else x.split("~")[1])

In [101]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                    Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1     Subsider Penjara (8 Bulan )\nPidana Denda Rp.1.000.000.000,00\nPidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )
2               Subsider Penjara (6 Bulan )\nPidana Denda Rp.800.000.000,00\nPidana Penjara Waktu Tertentu (6 Tahun )
3                                                                            Pidana Penjara Waktu Tertentu (2 Tahun )
4                                                                           Pidana Penjara Waktu Tertentu (11 Bulan )
5                Subsider Kurungan (1 Bulan )\nPidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Bulan )
6       Subsider Penjara (6 Bulan )\nPidana Denda Rp.800.000.000,00\nPidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
7                                                                            Pidana Penjara Waktu Tertentu (6 Bulan )
8     Subsider Penjara (6 Bulan )\nPidana Denda Rp.1.000

#### Convert to List

In [102]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['cleaned_putusan_hukuman'].str.split('\n')

In [103]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                    [Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )]
1     [Subsider Penjara (8 Bulan ), Pidana Denda Rp.1.000.000.000,00, Pidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )]
2               [Subsider Penjara (6 Bulan ), Pidana Denda Rp.800.000.000,00, Pidana Penjara Waktu Tertentu (6 Tahun )]
3                                                                            [Pidana Penjara Waktu Tertentu (2 Tahun )]
4                                                                           [Pidana Penjara Waktu Tertentu (11 Bulan )]
5                [Subsider Kurungan (1 Bulan ), Pidana Denda Rp.5.000.000,00, Pidana Penjara Waktu Tertentu (1 Bulan )]
6       [Subsider Penjara (6 Bulan ), Pidana Denda Rp.800.000.000,00, Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )]
7                                                                            [Pidana Penjara Waktu Tertentu (6 Bulan )]
8     [Subsider Penjara (6 Bulan ), Pida

#### Extract Pidana Penjara and Pidana Kurungan Text

In [107]:
df_clean_penuntut_exploded['putusan_hukuman']

0      I KOMANG ARIMBAWA Alias LELANG~Pidana Penjara ...
1      MOH. SOLEH Alias SOLEH~Subsider Penjara (8 Bul...
2      SOIMUN Alias WAYAN~Subsider Penjara (6 Bulan )...
3      I NENGAH SEMIARTA~Pidana Penjara Waktu Tertent...
4      I KETUT KERTA~Pidana Penjara Waktu Tertentu (1...
                             ...                        
546    I NYOMAN DUDUG~Pidana Penjara Waktu Tertentu (...
547    I MADE SARI~Pidana Penjara Waktu Tertentu (3 B...
548    I KETUT ARNYANA Als MASEM~Pidana Penjara Waktu...
549    I MADE SUGIARTA~Pidana Penjara Waktu Tertentu ...
550    I NENGAH APRI AGUSTINA ALS. degle als. GRANDON...
Name: putusan_hukuman, Length: 551, dtype: object

In [108]:
pattern = r'Pidana Penjara'
pattern_2 = r'Pidana Kurungan'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']
    
    for putusan_hukuman in list_putusan_hukuman:
        match = re.search(pattern, putusan_hukuman, re.IGNORECASE)
        match_2 = re.search(pattern_2, putusan_hukuman, re.IGNORECASE)

        if match or match_2:
            df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
            # print(f"{index} -> {putusan_hukuman}")

#### Check Null Extracted Putusan Hukuman

In [111]:
pd.set_option('display.max_colwidth', None)
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
print(null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']])
pd.reset_option('display.max_colwidth')

              nomor_perkara                         klasifikasi_perkara  \
139  40/Pid.Sus/2022/PN Amp                                   Kesehatan   
253  73/Pid.Sus/2021/PN Amp                                   Kesehatan   
264  20/Pid.Sus/2021/PN Amp                                   Narkotika   
266    16/Pid.B/2021/PN Amp                                Penganiayaan   
267    16/Pid.B/2021/PN Amp                                Penganiayaan   
330    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
331    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
332    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
333    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
334    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
335    30/Pid.B/2020/PN Amp                         Kejahatan Perjudian   
354  38/Pid.Sus/2019/PN Amp           Pertambangan Mineral dan Batubara   
358   9/Pid.Sus/2019/PN A

#### Use Subsider Penjara to Fill Null Values

In [112]:
pattern = r'Subsider'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']

    if row['extracted_putusan_hukuman'] is np.nan:
        for putusan_hukuman in list_putusan_hukuman:
            match = re.search(pattern, putusan_hukuman, re.IGNORECASE)

            if match:
                df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
                # print(f"{index} -> {putusan_hukuman}")
    

In [113]:
df_clean_penuntut_exploded[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']].head(10)

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
0,51/Pid.B/2024/PN Amp,Pencurian,[Pidana Penjara Waktu Tertentu (1 Tahun 6 Bula...,Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
1,43/Pid.Sus/2024/PN Amp,Narkotika,"[Subsider Penjara (8 Bulan ), Pidana Denda Rp....",Pidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )
2,40/Pid.Sus/2024/PN Amp,Narkotika,"[Subsider Penjara (6 Bulan ), Pidana Denda Rp....",Pidana Penjara Waktu Tertentu (6 Tahun )
3,49/Pid.B/2024/PN Amp,Pencurian,[Pidana Penjara Waktu Tertentu (2 Tahun )],Pidana Penjara Waktu Tertentu (2 Tahun )
4,45/Pid.B/2024/PN Amp,Pencurian,[Pidana Penjara Waktu Tertentu (11 Bulan )],Pidana Penjara Waktu Tertentu (11 Bulan )
5,39/Pid.Sus-LH/2024/PN Amp,Kerusakan Lingkungan Akibat Kegiatan Pertamban...,"[Subsider Kurungan (1 Bulan ), Pidana Denda Rp...",Pidana Penjara Waktu Tertentu (1 Bulan )
6,37/Pid.Sus/2024/PN Amp,Narkotika,"[Subsider Penjara (6 Bulan ), Pidana Denda Rp....",Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
7,36/Pid.B/2024/PN Amp,Pencurian,[Pidana Penjara Waktu Tertentu (6 Bulan )],Pidana Penjara Waktu Tertentu (6 Bulan )
8,25/Pid.Sus/2024/PN Amp,Narkotika,"[Subsider Penjara (6 Bulan ), Pidana Denda Rp....",Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )
9,25/Pid.Sus/2024/PN Amp,Narkotika,"[Subsider Penjara (6 Bulan ), Pidana Denda Rp....",Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )


In [114]:
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']]

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
253,73/Pid.Sus/2021/PN Amp,Kesehatan,[Lain-lain],NaN
358,9/Pid.Sus/2019/PN Amp,Lalu Lintas,"[Pidana Denda Rp.4.000.000,00, Lain-lain]",NaN
371,27/Pid.Sus/2018/PN Amp,Lalu Lintas,[Pidana Bersyarat],NaN
391,10/Pid.Sus/2020/PN Amp,Narkotika,[Lain-lain],NaN
413,8/Pid.B/2019/PN Amp,Kejahatan Perjudian,"[Pidana Denda Rp.5.000.000,00, Lain-lain]",NaN
465,45/Pid.Sus/2019/PN Amp,Lalu Lintas,"[Pidana Denda Rp.5.000.000,00, Lain-lain]",NaN
486,22/Pid.Sus/2018/PN Amp,Tindak Pidana Senjata Api atau Benda Tajam,[Lain-lain],NaN
525,10/Pid.Sus/2019/PN Amp,Lalu Lintas,"[Pidana Denda Rp.5.000.000,00]",NaN


#### Drop Null Values

In [115]:
df_clean_penuntut_exploded.dropna(subset=['extracted_putusan_hukuman'], inplace=True)

In [116]:
df_clean_penuntut_exploded.isna().sum()

nomor_perkara                0
klasifikasi_perkara          0
penuntut_umum                0
hakim                        0
jumlah_saksi                 0
cleaned_barang_bukti         0
cleaned_dakwaan              0
terdakwa                     0
putusan_hukuman              0
cleaned_putusan_hukuman      0
extracted_putusan_hukuman    0
dtype: int64

#### Extract Lama Kurungan Penjara

In [117]:
df_extracted = df_clean_penuntut_exploded.copy()

pattern_tbh = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tb = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*\)?"
pattern_th = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Hari\s*\)?"
pattern_bh = r"\(?\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tahun = r"\s*(\d+)\s*Tahun"
pattern_bulan = r"\s*(\d+)\s*Bulan"
pattern_hari = r"\s*(\d+)\s*Hari"

df_extracted['Tahun'] = None
df_extracted['Bulan'] = None
df_extracted['Hari'] = None

for index, row in df_extracted.iterrows():
    text = row["extracted_putusan_hukuman"]

    match_tbh = re.search(pattern_tbh, text, re.IGNORECASE)
    match_tb = re.search(pattern_tb, text, re.IGNORECASE)
    match_th = re.search(pattern_th, text, re.IGNORECASE)
    match_bh = re.search(pattern_bh, text, re.IGNORECASE)
    match_tahun = re.search(pattern_tahun, text, re.IGNORECASE)
    match_bulan = re.search(pattern_bulan, text, re.IGNORECASE)
    match_hari = re.search(pattern_hari, text, re.IGNORECASE)

    if match_tbh:
        tahun = int(match_tbh.group(1))
        bulan = int(match_tbh.group(2))
        hari = int(match_tbh.group(3))
        print(f"MATCH TBH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tb:
        tahun = int(match_tb.group(1))
        bulan = int(match_tb.group(2))
        hari = 0
        print(f"MATCH TB PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_th:
        tahun = int(match_th.group(1))
        bulan = 0
        hari = int(match_th.group(2))
        print(f"MATCH TH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bh:
        tahun = 0
        bulan = int(match_bh.group(1))
        hari = int(match_bh.group(2))
        print(f"MATCH BH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tahun:
        tahun = int(match_tahun.group(1))
        bulan = 0
        hari = 0
        print(f"MATCH TAHUN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bulan:
        tahun = 0
        bulan = int(match_bulan.group(1))
        hari = 0
        print(f"MATCH BULAN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_hari:
        tahun = 0
        bulan = 0
        hari = int(match_hari.group(1))
        print(f"MATCH HARI PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    else:
        tahun = None
        bulan = None
        hari = None

    df_extracted.loc[index, 'Tahun'] = tahun
    df_extracted.loc[index, 'Bulan'] = bulan
    df_extracted.loc[index, 'Hari'] = hari

MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan ) -> 1 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (5 Tahun 8 Bulan ) -> 5 Tahun, 8 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (6 Tahun ) -> 6 Tahun, 0 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (2 Tahun ) -> 2 Tahun, 0 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (11 Bulan ) -> 0 Tahun, 11 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (1 Bulan ) -> 0 Tahun, 1 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan ) -> 4 Tahun, 6 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (6 Bulan ) -> 0 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan ) -> 5 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan ) -> 5 Tahun, 6 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (6 Tahun ) 

In [118]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(df_extracted[["extracted_putusan_hukuman", "Tahun", "Bulan", "Hari"]])

pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

                             extracted_putusan_hukuman Tahun Bulan  Hari
0     Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )     1     6     0
1     Pidana Penjara Waktu Tertentu (5 Tahun 8 Bulan )     5     8     0
2             Pidana Penjara Waktu Tertentu (6 Tahun )     6     0     0
3             Pidana Penjara Waktu Tertentu (2 Tahun )     2     0     0
4            Pidana Penjara Waktu Tertentu (11 Bulan )     0    11     0
5             Pidana Penjara Waktu Tertentu (1 Bulan )     0     1     0
6     Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )     4     6     0
7             Pidana Penjara Waktu Tertentu (6 Bulan )     0     6     0
8     Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )     5     6     0
9     Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )     5     6     0
10            Pidana Penjara Waktu Tertentu (6 Tahun )     6     0     0
11            Pidana Penjara Waktu Tertentu (9 Bulan )     0     9     0
12            Pidana Penjara Waktu Tertentu (2 Tahu

#### Check Null Values after Extracting Lama Penjara

In [119]:
df_extracted.isnull().any()

nomor_perkara                False
klasifikasi_perkara          False
penuntut_umum                False
hakim                        False
jumlah_saksi                 False
cleaned_barang_bukti         False
cleaned_dakwaan              False
terdakwa                     False
putusan_hukuman              False
cleaned_putusan_hukuman      False
extracted_putusan_hukuman    False
Tahun                         True
Bulan                         True
Hari                          True
dtype: bool

#### Drop Null Values (TEMPP!!)

In [120]:
# Check for NaN values in 'extracted_duration' and drop the rows
# df_extracted = df_extracted.dropna(subset=['extracted_duration'])
# df_extracted = df_extracted.dropna(subset=['cleaned_barang_bukti']) # TEMPORARY
df_extracted.dropna(subset=['Tahun', 'Bulan', 'Hari'], inplace=True)

In [121]:
df_extracted.isnull().any()

nomor_perkara                False
klasifikasi_perkara          False
penuntut_umum                False
hakim                        False
jumlah_saksi                 False
cleaned_barang_bukti         False
cleaned_dakwaan              False
terdakwa                     False
putusan_hukuman              False
cleaned_putusan_hukuman      False
extracted_putusan_hukuman    False
Tahun                        False
Bulan                        False
Hari                         False
dtype: bool

#### Convert Kurungan Penjara menjadi Bulan

In [122]:
df_extracted['total_pidana_penjara_bulan'] = (df_extracted['Tahun'] * 12) + df_extracted['Bulan'] + (df_extracted['Hari'] / 30)

In [123]:
pd.set_option('display.max_rows', None)
print(df_extracted[["Tahun", "Bulan", "Hari", "total_pidana_penjara_bulan"]])
pd.reset_option('display.max_rows')

    Tahun Bulan Hari total_pidana_penjara_bulan
0       1     6    0                       18.0
1       5     8    0                       68.0
2       6     0    0                       72.0
3       2     0    0                       24.0
4       0    11    0                       11.0
5       0     1    0                        1.0
6       4     6    0                       54.0
7       0     6    0                        6.0
8       5     6    0                       66.0
9       5     6    0                       66.0
10      6     0    0                       72.0
11      0     9    0                        9.0
12      2     0    0                       24.0
13      0     1    0                        1.0
14      0     8    0                        8.0
15      0     8    0                        8.0
16      0     8    0                        8.0
17      0     3    0                        3.0
18      0     9    0                        9.0
19      5     6    0                    

In [124]:
df_extracted.to_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}_FINAL.csv', index=False)

## Data Validation

In [125]:
df_stage_1 = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}_FINAL.csv')
df_stage_2 = pd.read_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}_FINAL.csv')

In [126]:
print(df_stage_1.columns)
print(df_stage_2.columns)

Index(['status_perkara', 'nomor_perkara', 'klasifikasi_perkara',
       'tanggal_pendaftaran', 'lama_proses', 'terdakwa', 'penuntut_umum',
       'hakim', 'jumlah_saksi', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti', 'cleaned_dakwaan'],
      dtype='object')
Index(['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa',
       'putusan_hukuman', 'cleaned_putusan_hukuman',
       'extracted_putusan_hukuman', 'Tahun', 'Bulan', 'Hari',
       'total_pidana_penjara_bulan'],
      dtype='object')


In [127]:
df_validation = df_stage_2[['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'total_pidana_penjara_bulan']]

In [128]:
df_validation.head(5)

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,51/Pid.B/2024/PN Amp,Pencurian,Ida Ayu Putu Widhiantini,Ayu Putri Cempakasari,5,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...,DAKWAAN. - Bahwa ia Terdakwa I KOMANG ARIMBAWA...,I KOMANG ARIMBAWA Alias LELANG,18.0
1,43/Pid.Sus/2024/PN Amp,Narkotika,"Ardi Putra Dewa Agung, S.H",Meilia Christina Mulyaningrum,2,4 (empat) paket narkotika jenis shabu dengan b...,Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alia...,MOH. SOLEH Alias SOLEH,68.0
2,40/Pid.Sus/2024/PN Amp,Narkotika,"M. Thoriq Ardiansyah, S.H.",Ayu Putri Cempakasari,3,2 paket klip plastic berisi serbuk kristal ben...,"DAKWAAN. - Bahwa Terdakwa SOIMUN alias WAYAN, ...",SOIMUN Alias WAYAN,72.0
3,49/Pid.B/2024/PN Amp,Pencurian,"Ardi Putra Dewa Agung, S.H",Ayu Putri Cempakasari,2,satu lembar STNK dari satu unit sepeda motor m...,Dakwaan. Bahwa Terdakwa I Nengah Semiarta (yan...,I NENGAH SEMIARTA,24.0
4,45/Pid.B/2024/PN Amp,Pencurian,Ida Ayu Putu Widhiantini,Meilia Christina Mulyaningrum,4,1 (satu) buah toples plastik bening dengan tut...,DAKWAAN. - Bahwa Terdakwa I KETUT KERTA (selan...,I KETUT KERTA,11.0


#### Validate Data Types

In [129]:
for column in df_validation.columns:
    types_in_column = df_stage_2[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara' -> [<class 'str'>]
Data types in column 'penuntut_umum' -> [<class 'str'>]
Data types in column 'hakim' -> [<class 'str'>]
Data types in column 'jumlah_saksi' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan' -> [<class 'str'>]
Data types in column 'terdakwa' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]


#### Merge DataFrame

In [130]:
merged_df = df_validation.merge(df_stage_1, on='nomor_perkara', suffixes=('_stage_2', '_stage_1'))

In [131]:
for column in merged_df.columns:
    types_in_column = merged_df[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_2' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_2' -> [<class 'str'>]
Data types in column 'hakim_stage_2' -> [<class 'str'>]
Data types in column 'jumlah_saksi_stage_2' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti_stage_2' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan_stage_2' -> [<class 'str'>]
Data types in column 'terdakwa_stage_2' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]
Data types in column 'status_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_1' -> [<class 'str'>]
Data types in column 'tanggal_pendaftaran' -> [<class 'str'>]
Data types in column 'lama_proses' -> [<class 'str'>]
Data types in column 'terdakwa_stage_1' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_1' -> [<class 'str'>]
Data types in column 'hakim_stage_1' -> [<class 'str'>]
Da

In [132]:
merged_df.isna().sum()

nomor_perkara                   0
klasifikasi_perkara_stage_2     0
penuntut_umum_stage_2           0
hakim_stage_2                   0
jumlah_saksi_stage_2            0
cleaned_barang_bukti_stage_2    0
cleaned_dakwaan_stage_2         0
terdakwa_stage_2                0
total_pidana_penjara_bulan      0
status_perkara                  0
klasifikasi_perkara_stage_1     0
tanggal_pendaftaran             0
lama_proses                     0
terdakwa_stage_1                0
penuntut_umum_stage_1           0
hakim_stage_1                   0
jumlah_saksi_stage_1            0
putusan_hukuman                 0
barang_bukti                    0
dakwaan                         0
cleaned_barang_bukti_stage_1    0
cleaned_dakwaan_stage_1         0
dtype: int64

#### Validate Each Row Based on Nomor Perkara (Klasifikasi Perkara, Terdakwa, Penuntut Umum, Hakim, Jumlah Saksi)

##### Klasifikasi Perkara

In [133]:
merged_df['match_klasifikasi_perkara'] = merged_df['klasifikasi_perkara_stage_2'] == merged_df['klasifikasi_perkara_stage_1']

for _, row in merged_df.iterrows():
    if row['match_klasifikasi_perkara']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: KLASIFIKASI PERKARA MISMATCH!")

##### Terdakwa

In [134]:
for index, row in merged_df.iterrows():
    terdakwa_stage_2 = row['terdakwa_stage_2']

    if terdakwa_stage_2 in row['terdakwa_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: TERDAKWA MISMATCH!")

##### Penuntut Umum

In [135]:
for index, row in merged_df.iterrows():
    penuntut_umum_stage_2 = row['penuntut_umum_stage_2']

    if penuntut_umum_stage_2 in row['penuntut_umum_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: PENUNTUT UMUM MISMATCH!")


##### Hakim

In [136]:
for index, row in merged_df.iterrows():
    hakim_stage_2 = row['hakim_stage_2']

    if hakim_stage_2 in row['hakim_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: HAKIM MISMATCH!")

##### Jumlah Saksi

In [137]:
for index, row in merged_df.iterrows():
    if row['jumlah_saksi_stage_2'] == row['jumlah_saksi_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: JUMLAH SAKSI MISMATCH!")

#### Drop Unnecessary Columns

In [138]:
merged_df.columns

Index(['nomor_perkara', 'klasifikasi_perkara_stage_2', 'penuntut_umum_stage_2',
       'hakim_stage_2', 'jumlah_saksi_stage_2', 'cleaned_barang_bukti_stage_2',
       'cleaned_dakwaan_stage_2', 'terdakwa_stage_2',
       'total_pidana_penjara_bulan', 'status_perkara',
       'klasifikasi_perkara_stage_1', 'tanggal_pendaftaran', 'lama_proses',
       'terdakwa_stage_1', 'penuntut_umum_stage_1', 'hakim_stage_1',
       'jumlah_saksi_stage_1', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti_stage_1', 'cleaned_dakwaan_stage_1',
       'match_klasifikasi_perkara'],
      dtype='object')

In [139]:
df_stage_3 = df_validation[['klasifikasi_perkara', 
                            'penuntut_umum', 
                            'hakim', 
                            'jumlah_saksi', 
                            'cleaned_barang_bukti', 
                            'cleaned_dakwaan', 
                            'terdakwa', 
                            'total_pidana_penjara_bulan']]

df_stage_3.to_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_{database_name}_FINAL.csv', index=False)

#### Check Saved CSV

In [140]:
df_stage_3 = pd.read_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_{database_name}_FINAL.csv')

In [141]:
df_stage_3.head(5)

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,Pencurian,Ida Ayu Putu Widhiantini,Ayu Putri Cempakasari,5,1(satu) lembar Surat Tanda Nomor Kendaraan Nom...,DAKWAAN. - Bahwa ia Terdakwa I KOMANG ARIMBAWA...,I KOMANG ARIMBAWA Alias LELANG,18.0
1,Narkotika,"Ardi Putra Dewa Agung, S.H",Meilia Christina Mulyaningrum,2,4 (empat) paket narkotika jenis shabu dengan b...,Dakwaan. Kesatu. Bahwa Terdakwa Moh Soleh Alia...,MOH. SOLEH Alias SOLEH,68.0
2,Narkotika,"M. Thoriq Ardiansyah, S.H.",Ayu Putri Cempakasari,3,2 paket klip plastic berisi serbuk kristal ben...,"DAKWAAN. - Bahwa Terdakwa SOIMUN alias WAYAN, ...",SOIMUN Alias WAYAN,72.0
3,Pencurian,"Ardi Putra Dewa Agung, S.H",Ayu Putri Cempakasari,2,satu lembar STNK dari satu unit sepeda motor m...,Dakwaan. Bahwa Terdakwa I Nengah Semiarta (yan...,I NENGAH SEMIARTA,24.0
4,Pencurian,Ida Ayu Putu Widhiantini,Meilia Christina Mulyaningrum,4,1 (satu) buah toples plastik bening dengan tut...,DAKWAAN. - Bahwa Terdakwa I KETUT KERTA (selan...,I KETUT KERTA,11.0


In [142]:
df_stage_3.isna().sum()

klasifikasi_perkara           0
penuntut_umum                 0
hakim                         0
jumlah_saksi                  0
cleaned_barang_bukti          0
cleaned_dakwaan               0
terdakwa                      0
total_pidana_penjara_bulan    0
dtype: int64

# Text Summarization

In [5]:
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)

CUDA available: True
Device count: 1
CUDA version: 12.1


In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA devices available.")

In [4]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1214, 0.0298, 0.0406],
        [0.8481, 0.8833, 0.1258],
        [0.8622, 0.3680, 0.7690],
        [0.6251, 0.0697, 0.8667],
        [0.2934, 0.7982, 0.9171]])


In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Sep_12_02:55:00_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.77
Build cuda_12.6.r12.6/compiler.34841621_0


In [16]:
!nvidia-smi

Fri Dec  6 12:56:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              4W /   76W |     670MiB /   8188MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Text Summarization "t5-base-indonesian-summarization-cased"

In [5]:
from transformers import pipeline

from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
import nltk

import time
from torch import float16

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### CLEAN \N

In [6]:
df_extracted['cleaned_dakwaan'] = df_extracted['cleaned_dakwaan'].str.replace('\n', ' ', regex=True)

In [9]:
df_extracted['cleaned_dakwaan']

0       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
1       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
2       DAKWAAN   KESATU             Bahwa ia Terdakwa...
3       DAKWAAN   KESATU                              ...
4       Dakwaan Ke Satu - Bahwa  terdakwa  SANG PUTU W...
                              ...                        
1191    Kesatu  Pasal 83 ayat (1) huruf b UU RI No. 18...
1192    Perbuatan terdakwa sebagaimana diatur dan dian...
1193    Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1194    Pertama  Pasal 187 ayat (1) KUHP dan Kedua  Pa...
1195          Pasal 362 Kitab Undang-undang Hukum Pidana.
Name: cleaned_dakwaan, Length: 1196, dtype: object

In [ ]:
model_names = [
    "cahya/bert2bert-indonesian-summarization",
    "rowjak/bert-indonesian-news-summarization"
    "cahya/bert2gpt-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
]

In [ ]:


pipelines = {
    model_name: pipeline(
        "summarization",
        model=model_name,
        use_fast=True,
        device="cuda",
        batch_size=batch_size,
        torch_dtype="auto"
    )
    for model_name in model_names
}

In [ ]:
import warnings
from transformers import pipeline, AutoTokenizer
pipelines ={}

for model_name in model_names:
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,
    )

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch: Sentencepiece tokenizer fallback detected. Using slow tokenizer.
Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


The encoder model config class: <class 'transformers.models.bert.configuration_bert.BertConfig'> is different from the decoder model config class: <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>. It is not recommended to use the `AutoTokenizer.from_pretrained()` method in this case. Please use the encoder and decoder specific tokenizer classes.
Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

In [20]:
pipelines

{'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243ddf0ece0>,
 'panggi/t5-small-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e05602e0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d667f610>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e056dae0>,
 'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbfd0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbe20>,
 'rayendito/mt5-small-finetuned-xl-sum-indonesia': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d38195d0>,


In [17]:
import pprint

pprint.pprint(pipelines)

{'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243E04B5EA0>,
 'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D3818FA0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E7FA30>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D6723010>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E3DCC0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4BB4880>,
 'panggi/t5-small-indonesian-summ

In [8]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48
...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24


In [10]:
avg_length = df_extracted['cleaned_dakwaan'].apply(lambda text: len(text.split())).mean()

print(f"Average length of cleaned_dakwaan: {avg_length:.2f} words")

Average length of cleaned_dakwaan: 691.86 words


In [9]:
import datetime
import psutil
import os
def log_memory_usage():
    print(f"CPU usage: {psutil.cpu_percent()}%, Memory usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
        print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB")

save_dir = '../Data/STAGE 3 PREPROCESSING'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)



In [10]:
import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", message="Your max_length is set to \d+, but your input_length is only \d+", category=UserWarning, module="transformers")
chunk_size = 450
batch_size = 12

for model_name in model_names:
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True).to('cuda')

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            if index % 100 == 0:
                print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
                    batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
                else:
                    batch_max_length = 100  # Default max for very short chunks
                    batch_min_length = 50  # Default min for very short chunks

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipe
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Fast tokenizer not supported for panggi/t5-base-indonesian-summarization-cased due to error: 'T5TokenizerFast' object has no attribute 'to'. Using slow tokenizer.
Processing row 1/1196 - 100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

Processing row 101/1196 - 200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

...

Processing row 1001/1196 - 1100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your m

Processing row 1101/1196 - 1200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241208_055637.csv
Model panggi/t5-base-indonesian-summarization-cased - Execution time: 20091.55 seconds



In [100]:
df_tempor = pd.read_csv('../Data/STAGE 3 PREPROCESSING/Final Singaraja Summarization.csv')

In [108]:
df_tempor.columns

Index(['Unnamed: 0', 'klasifikasi_perkara', 'terdakwa', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan',
       'total_pidana_penjara_bulan',
       'panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan',
       'rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan',
       'panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan',
       'cahya_bert2bert_summarization_dakwaan',
       'rowjak/bert-indonesian-news-summarization',
       'cahya_bert2gpt_summarization_dakwaan',
       'cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan'],
      dtype='object')

In [107]:
df_tempor.iloc[:, -12:]

,cleaned_dakwaan,total_pidana_penjara_bulan,panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan,rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan,panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan,cahya_bert2bert_summarization_dakwaan,rowjak/bert-indonesian-news-summarization,cahya_bert2gpt_summarization_dakwaan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subagia...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subuku ...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Kepolisian KESATU Bahwa ia Terdakwa I NYOMAN S...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Seorang pria terdakwa seorang pria yang terdak...,DKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUART...,Bahwa selanjutnya dilakukan penimban...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa terdakwa i nyoman suarta ...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,Kedakwaan KESATU Bahwa ia Terdakwa RICO JAYADI...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,"Seorang warga Banjar, RICOJAYADI, terdakwa RIC...",DKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI pa...,Plastik klip berisi butiran kristal ...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...,Ke Satu - Bahwa terdakwa SANG PUTU WIDIANA ber...,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,"Seorang terdakwa SANG PUTU WIDIANA, seorang sa...",Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,Bahwa terhadap barang bukti berupa 6...,dakwaan ke satu - satunya tentang terdakwa san...,dakwaan ke satu - satunya ditujukan terhadap t...,dakwaan terhadap terdakwa sang putu widiana be...,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1191,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat ( 1 ) huruf b UU RI No. 1...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,Kesatu Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Sebuah Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,NaN,kesatu pasal 83 ayat ( 1 ) huruf b uu ri no. 1...,dua pasal dalam uu ri nomor 18 tahun 2013 tent...,kesatu pasal 83 ayat ( 1 ) huruf b undang - un...,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Perbuatan terdakwa sebagaimana diatur da

# single model


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 16  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2bert_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model {model_name} at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


OSError: Cannot save file into a non-existent directory: '..\Data\STAGE_3_PREPROCESSING'

In [12]:
# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model cahya/bert2bert at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")


CHECKPOINT! Data Saved for model cahya/bert2bert at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_101016.csv
Summarization completed in 8207.40 seconds


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("rowjak/bert-indonesian-news-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("rowjak/bert-indonesian-news-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],
                    num_beams=2,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    max_length=batch_max_length,
                    min_length=batch_min_length)

                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['rowjak/bert-indonesian-news-summarization'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


NameError: name 'model_name' is not defined

In [14]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_105235.csv


In [15]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2gpt-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2gpt-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2gpt_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "_name_or_path": "cahya/gpt2-small-indonesian-522M",
  "activation_func

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196

CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_122142.csv
Summarization completed in 5332.07 seconds


In [27]:
del encoded_input
del summary_ids
del model
del tokenizer
torch.cuda.empty_cache()

NameError: name 'summary_ids' is not defined

In [ ]:
# dont run!

import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

chunk_size = 450
batch_size = 2048

for model_name in model_names:
    break
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # Print progress every 100 rows
            print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            # if index % 50 == 0:
                # print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 50}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = max(min(max(length // 3 for length in valid_lengths), 150), 30)
                    batch_min_length = max(batch_max_length // 2, 20)
                else:
                    batch_max_length = 30
                    batch_min_length = 20

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE_3_PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model {key} at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipes
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Processing row 1/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 2/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 3/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Processing row 4/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


...

Your max_length is set to 30, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Processing row 24/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


Processing row 25/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 26/1196 for model cahya/t5-base-indonesian-summarization-cased


: 

In [11]:
del pipe
torch.cuda.empty_cache()

In [12]:

del pipelines
torch.cuda.empty_cache()

In [24]:
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)

In [22]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4,Perbuatan terdakwa sebagaimana diatur dan dian...
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9,Pasal 351 ayat( 1) Kitab Undang-undang Hukum p...
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24,Pasal 187 ayat(1) KUHP dan Kedua Pasal 363 aya...


In [ ]:
pipe = pipeline("text2text-generation", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=2)

# Track the start time
start_time = time.time()

# Initialize an empty list to store the summaries
summarized_texts = []

# Iterate through the rows of the DataFrame
for index, row in df_extracted.iterrows():
    text = row['cleaned_dakwaan']

    # Chunk the text into smaller pieces
    chunk_size = 512
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    try:
        # Summarize each chunk
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    # Combine the summarized chunks into one string
    full_summary = " ".join(summaries)

    # Append the summarized text to the list
    summarized_texts.append(full_summary)

# Add the summarized texts as a new column
df_extracted['second_model_summarization_dakwaan'] = summarized_texts

# Print execution time
print(f"Execution time: {time.time() - start_time} seconds")


### DOCS

In [524]:
from transformers import pipeline

# pipe = pipeline("text2text-generation", model="facebook/mbart-large-50", use_fast=False, device="cuda", batch_size=2)
# pipe = pipeline("text2text-generation", model="indobenchmark/indobart-large")
# pipe = pipeline("summarization", model="mrm8488/t5-base-finetuned-indonesian-summarization")
# pipe = pipeline("text2text-generation", model="google/mt5-large", use_fast=False)

# tensorflow_summarization_models = [
#     "thonyyy/pegasus_indonesian_base-finetune",
#     "thonyyy/pegasus_indonesian_base-pretrain"
# ]

pytorch_summarization_models = [
    # "cahya/bert2bert-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    # "cahya/bert2gpt-indonesian-summarization",
    "panggi/t5-base-indonesian-summarization-cased",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
    # "rowjak/bert-indonesian-news-summarization"
]


# tensorflow_summarization_pipes = [
#     pipeline("summarization", model=model, from_tf=True, device=0, batch_size=4)
#     for model in tensorflow_summarization_models
# ]

In [ ]:
def evaluate_meteor(generated, reference):
    # Tokenize the generated and reference text
    tokenized_generated = word_tokenize(generated)
    tokenized_reference = word_tokenize(reference)
    
    # Evaluate METEOR score
    score = meteor_score.meteor_score([tokenized_reference], tokenized_generated)
    return score

def evaluate_rouge(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

def evaluate_bleu(generated, reference):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    score = sentence_bleu([reference_tokens], generated_tokens)
    return score

# BERTScore evaluation
def evaluate_bertscore(generated, reference):
    P, R, F1 = score([generated], [reference], lang="en")
    return F1.item()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pipe = pipeline("summarization", model="cahya/bert2bert-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/bert2gpt-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rayendito/mt5-small-finetuned-xl-sum-indonesia", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rowjak/bert-indonesian-news-summarization", use_fast=False, device="cuda", batch_size=8)


In [ ]:
import time

text = df_extracted.loc[0,'cleaned_dakwaan']
print(len(text))
reference_summary = """Terdakwa Ketut Subagia dan Komang Aditya pada 2 Juni 2024 sekitar pukul 00.30 WITA di pesisir pantai Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng, Bali, diduga mengadakan permainan judi bola adil secara ilegal. Ketut Subagia bertindak sebagai penyelenggara dan pemodal, sementara Komang Aditya membantu sebagai pekerja yang merapikan uang taruhan serta menyerahkannya kepada pemenang. Permainan melibatkan papan bergambar, bola karet, perlak bergambar, dan cek kayu sebagai pengganti uang taruhan. Barang bukti yang disita meliputi papan bola adil, tas, perlak, bola karet, kayu penyeimbang, bedak bayi, cek kayu dalam berbagai denominasi, lap, dan uang tunai Rp1.108.000. Permainan tersebut dilakukan dengan sistem taruhan yang memberikan hadiah kelipatan sembilan kali lipat dari jumlah taruhan jika menang, namun peserta yang kalah kehilangan uangnya. Aktivitas ini berlangsung selama beberapa jam dengan melibatkan sekitar 20 pemain dan keuntungan digunakan untuk kebutuhan sehari-hari. Namun, kegiatan ini dilakukan tanpa izin resmi dan melanggar hukum, sehingga kedua terdakwa ditangkap dan diancam pidana berdasarkan Pasal 303 KUHP juncto Undang-Undang Nomor 7 Tahun 1974 tentang Penertiban Perjudian."""

for i, model in enumerate(pytorch_summarization_models):
    print(model)
    pipe = pipeline("summarization", model=model, device="cuda", batch_size=4, use_fast=False) 
    chunk_size = 512 

    start_time = time.time()

    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    try:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    full_summary = " ".join(summaries)
    end_time = time.time()

    time_taken = end_time - start_time

    print("------------------------------------------------------")

    rouge_scores = evaluate_rouge(full_summary, reference_summary)
    print("ROUGE Scores:", rouge_scores)
    meteor_score_value = evaluate_meteor(full_summary, reference_summary)
    print("METEOR Score:", meteor_score_value)
    bleu_score_value = evaluate_bleu(full_summary, reference_summary)
    print("BLEU Score:", bleu_score_value)
    bertscore_value = evaluate_bertscore(full_summary, reference_summary)
    print("BERTScore F1:", bertscore_value)

    # Display ROUGE scores
    print(len(full_summary))
    print(f"Summary: using {model}: {full_summary}")
    print("------------------------------------------------------")

    del pipe
    torch.cuda.empty_cache()

    print(f"Time taken for summarization: {time_taken:.2f} seconds\n\n")


21579
cahya/t5-base-indonesian-summarization-cased


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.12121212121212122, recall=0.5847953216374269, fmeasure=0.20080321285140565), 'rouge2': Score(precision=0.04611650485436893, recall=0.2235294117647059, fmeasure=0.07645875251509053), 'rougeL': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586), 'rougeLsum': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586)}
METEOR Score: 0.30761823728726106
BLEU Score: 0.017997592188981122


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8316876292228699
5486
Summary: using cahya/t5-base-indonesian-summarization-cased: DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada saat terdakwa KETUT SUBAGIA bersama dengan Terdakwa UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penya Dua buah bola bergambar, satu (satu) buah tas kain warna abu-abu, satu (satu) buah tas karung warna Sebanyak 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 (duapuluh), 17 (tujuh belas) KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola adil dan juga sebagai penyelenggara yang mengadakan permainan KOMANG ADITYA sebagai pekerja dan merapikan uang serta memberikan uang kepada pemenang permainan bola adil. Bahwa dalam permainan judi bola adil Pasangan uang taruhan judi bola adil 1 (satu)

...

BERTScore F1: 0.8033068180084229
176
Summary: using panggi/t5-base-indonesian-summarization-cased:                 Bahwa di papan bola adil terdapat 4 warna yaitu merah, kuning hijau dan hitam adapun gambarnya yaitu berbentuk segitiga, palang, bola,                          
------------------------------------------------------
Time taken for summarization: 17.71 seconds


rayendito/mt5-small-finetuned-xl-sum-indonesia
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.13314037626628075, recall=0.5380116959064327, fmeasure=0.2134570765661253), 'rouge2': Score(precision=0.043478260869565216, recall=0.17647058823529413, fmeasure=0.06976744186046512), 'rougeL': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181), 'rougeLsum': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181)}
METEOR Score: 0.3027963518759939
BLEU Score: 0.01799532323977946


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8162375688552856
4444
Summary: using rayendito/mt5-small-finetuned-xl-sum-indonesia: Seorang terdakwa kecelakaan pertama - telah mendakwa seorang terdakwa kecelakaan pertama - setelah Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan Seorang pemain bola adil di Bali telah berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Kec Sebuah tas kain warna hitam, 1 (satu) buah tas kain warna hitam, 1 (satu) buah tas Sebuah lembar uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari Seorang terdakwa keTUT SUBAGIA dalam permainan judi bola adil di Indonesia, telah mendakwa terdakwa keTUT Suba Pemenang permainan bola adil di Indonesia, telah menentukan kemenangan dalam permainan bola adil, karena seorang pemodal Permainan bola adil 1 (satu) buah tas karung warna biru garis merah digunakan untuk pasangan uang Sebuah papan cek kayu berwarna hitam digunakan untuk penyimpan bola adil setelah dig

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8330612778663635
4361
Summary: using interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai buah tas kain warna abu-abu, 1 (satu) buah perlak bergambar, 1 (satu) buah perlak uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari 8 (delapan Terdakwa KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola Pemodal, mengambil uang - uang taruhan yang berada di atas perlak, menggelindingkan bola, sedangkan PAN bola adil 1 (satu) buah perlak bergambar digunakan untuk pasangan uang taruhan bola adil 1 (satu s kain warna hitam digunakan untuk penyimpan b